# Libraries

This code block sets up the foundation for the Adaptive Polynomial Surrogate (APS) framework by importing all necessary dependencies and establishing GPU/CPU interoperability.

It begins with `__future__` annotations for forward compatibility and integrates the RAPIDS Memory Manager (RMM) CuPy allocator, ensuring efficient GPU memory handling. Standard Python libraries such as **os**, **math**, **time**, **warnings**, **re**, and **dataclasses** provide core functionality, while typing hints improve code readability and maintainability.

For array and dataframe processing, the block incorporates **CuPy** (a GPU-accelerated NumPy replacement) and **cuDF** (a GPU-based pandas equivalent), including its pandas-compatibility layer. Deep learning capabilities are enabled through **PyTorch**, with support for neural network modules, functional layers, and DLPack utilities that ensure interoperability between frameworks. Gradient-boosting models are brought in via **XGBoost** and **CatBoost**, while **cuML** provides polynomial feature engineering and GPU-accelerated train–test splitting.

To maintain cleaner execution, Python warnings are suppressed, and a type alias `_Array` is defined to standardize array-like inputs across CuPy and PyTorch. A unified namespace `xp` is set to CuPy, and two utility functions are introduced:  
- **asxp**, which ensures arrays are converted to the active backend (CuPy in this setup).  
- **kfold_folds**, which generates randomized k-fold cross-validation splits using multiple fallback RNG methods for reproducibility.

Finally, cuDF type-checking utilities (**is_numeric_dtype**, **is_datetime_dtype**, **is_bool_dtype**) are imported to support data validation during preprocessing.

Overall, this initialization block establishes the computational environment, backend consistency, and baseline functionality required for APS to operate seamlessly across GPUs and CPUs.


In [1]:
from __future__ import annotations

# --- RMM init FIRST (before any GPU allocations) ---
import rmm, cupy as cp
from rmm.allocators.cupy import rmm_cupy_allocator

# Safer pool sizing: ~80% of free VRAM at start
free_mem, total_mem = cp.cuda.runtime.memGetInfo()
pool_bytes = int(3072)
rmm.reinitialize(pool_allocator=True, initial_pool_size=pool_bytes)

# --- Arrays / DataFrames ---
cp.cuda.set_allocator(rmm_cupy_allocator)
cp.cuda.set_pinned_memory_allocator(cp.cuda.PinnedMemoryPool().malloc)
_ = cp.zeros(1, dtype=cp.float32)  # optional warmup

import cudf
from cudf.api.types import is_numeric_dtype, is_datetime_dtype, is_bool_dtype

# --- Torch ---
import torch
from rmm.allocators.torch import rmm_torch_allocator
if hasattr(torch.cuda, "memory") and hasattr(torch.cuda.memory, "change_current_allocator"):
    torch.cuda.memory.change_current_allocator(rmm_torch_allocator)

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.dlpack as torch_dlpack  # single DLPack import

# --- Tree baselines (GPU) ---
import xgboost as xgb
from catboost import CatBoostRegressor, Pool as CatPool

# --- RAPIDS helpers ---
from cuml.preprocessing import PolynomialFeatures as cuPolynomialFeatures
from cuml.model_selection import train_test_split as cu_train_test_split
from cuml.ensemble import RandomForestRegressor as cuRF

# --- Stdlib & typing ---
import os, math, time, warnings, re
from dataclasses import dataclass
from typing import Optional, Dict, Any, Tuple, Iterable, List, Union

warnings.filterwarnings("ignore")

# Types / interop
_Array = Union[cp.ndarray, torch.Tensor]
xp = cp

def asxp(a, dtype=None):
    return a if isinstance(a, cp.ndarray) else cp.asarray(a, dtype=dtype)

def kfold_folds(n: int, cv_folds: int, seed: int = 42):
    k = int(min(cv_folds, max(2, n // 10)))
    try:
        idx = cp.random.default_rng(seed).permutation(n)
    except Exception:
        try:
            idx = cp.random.RandomState(seed).permutation(n)
        except Exception:
            cp.random.seed(seed)
            idx = cp.random.permutation(n)
    return cp.array_split(idx, k)


# Polynomial Features for Generation

In [2]:
def gpu_poly_features(X_cp: cp.ndarray, degree: int, include_bias: bool = False):
    """
    Build polynomial features up to `degree` on GPU - SHARED FUNCTION
    """
    n, d = int(X_cp.shape[0]), int(X_cp.shape[1])
    
    # Pre-calculate total features
    n_features = d  # degree 1
    if degree >= 2:
        n_features += d + (d * (d - 1)) // 2
    if degree >= 3:
        n_features += d + d * (d - 1) + (d * (d - 1) * (d - 2)) // 6
    if include_bias:
        n_features += 1
    
    # Pre-allocate and fill in-place (from previous optimization)
    X_poly = cp.empty((n, n_features), dtype=X_cp.dtype)
    names = []
    idx = 0
    
    if include_bias:
        X_poly[:, idx] = 1.0
        names.append("1")
        idx += 1
    
    # degree 1
    for j in range(d):
        X_poly[:, idx] = X_cp[:, j]
        names.append(f"x{j}")
        idx += 1
    
    if degree >= 2:
        for j in range(d):
            X_poly[:, idx] = X_cp[:, j] ** 2
            names.append(f"x{j}^2")
            idx += 1
        for a in range(d):
            for b in range(a + 1, d):
                X_poly[:, idx] = X_cp[:, a] * X_cp[:, b]
                names.append(f"x{a} x{b}")
                idx += 1
    
    if degree >= 3:
        for j in range(d):
            X_poly[:, idx] = X_cp[:, j] ** 3
            names.append(f"x{j}^3")
            idx += 1
        for a in range(d):
            x_a_sq = X_cp[:, a] ** 2
            for b in range(d):
                if b == a:
                    continue
                X_poly[:, idx] = x_a_sq * X_cp[:, b]
                names.append(f"x{a}^2 x{b}")
                idx += 1
        for a in range(d):
            for b in range(a + 1, d):
                for c in range(b + 1, d):
                    X_poly[:, idx] = X_cp[:, a] * X_cp[:, b] * X_cp[:, c]
                    names.append(f"x{a} x{b} x{c}")
                    idx += 1
    
    return X_poly, names

# Fast CV Score

In [3]:
# Add this ONCE after the polynomial features function (around line 150)
def _fast_cv_score(X_cp, y_cp, cv_folds=3):
    """Fast CV R² using pre-allocated folds - SHARED FUNCTION"""
    n, p = int(X_cp.shape[0]), int(X_cp.shape[1])
    if p >= 0.8 * n or n < 15:
        return -10.0

    k = int(min(cv_folds, max(2, n // 10)))
    
    # Create folds ONCE and reuse indices
    cp.random.seed(42)
    idx = cp.arange(n)
    cp.random.shuffle(idx)  # In-place shuffle instead of permutation
    
    fold_size = n // k
    r2s = []
    
    for i in range(k):
        # Use slicing instead of fancy indexing where possible
        val_start = i * fold_size
        val_end = (i + 1) * fold_size if i < k - 1 else n
        
        # Create boolean mask for faster indexing
        val_mask = cp.zeros(n, dtype=bool)
        val_mask[val_start:val_end] = True
        train_mask = ~val_mask
        
        X_tr = X_cp[train_mask]
        y_tr = y_cp[train_mask]
        X_va = X_cp[val_mask]
        y_va = y_cp[val_mask]
        
        # Fast linear regression
        lr = TorchLinearRegression(fit_intercept=True, device="cuda")
        lr.fit(X_tr, y_tr)
        y_hat = lr.predict(X_va)
        
        y_mean = cp.mean(y_va, axis=0, keepdims=True)
        ss_res = cp.sum((y_va - y_hat) ** 2, axis=0)
        ss_tot = cp.sum((y_va - y_mean) ** 2, axis=0) + 1e-12
        r2 = 1.0 - (ss_res / ss_tot)
        r2_scalar = cp.mean(r2)
        
        if bool(cp.isfinite(r2_scalar).item()):
            r2s.append(r2_scalar)
    
    return float(cp.mean(cp.asarray(r2s, dtype=cp.float32)).item()) if r2s else -10.0

# Forward Selection

In [4]:
def _vectorized_forward_selection(X_cand, y_cp, tolerance=1e-4, verbose=False):
    """Greedy forward selection using GPU-backed CV R² - SHARED FUNCTION"""
    if X_cand.shape[1] == 0:
        return []

    n_candidates = X_cand.shape[1]
    selected = []
    remaining = list(range(n_candidates))
    best_score = -cp.inf
    no_improve = 0

    # More aggressive limits (from previous optimization)
    n = int(X_cand.shape[0])
    if n < 15:
        return selected

    while (remaining and no_improve < 2 and len(selected) < min(15, n_candidates) 
           and len(selected) < n // 10):
        
        scores = _batch_evaluate_candidates(X_cand, y_cp, selected, remaining)
        if len(scores) == 0:
            break

        best_idx = int(cp.argmax(scores))
        best_cand = remaining[best_idx]
        cand_score = float(scores[best_idx])

        if cand_score > best_score + tolerance:
            selected.append(best_cand)
            remaining.pop(best_idx)
            best_score = cand_score
            no_improve = 0
            if verbose:
                print(f"Selected feature {best_cand}, CV R²={best_score:.4f}")
        else:
            no_improve += 1

    return selected

def _batch_evaluate_candidates(X_cand, y_cp, selected, remaining):
    """Evaluate batch of candidates - SHARED FUNCTION"""
    scores = []
    X_sel = X_cand[:, selected] if selected else None

    for cand in remaining:
        X_test = X_cand[:, [cand]] if X_sel is None else cp.concatenate([X_sel, X_cand[:, [cand]]], axis=1)
        score = _fast_cv_score(X_test, y_cp, cv_folds=3)
        scores.append(score)

    return cp.asarray(scores, dtype=cp.float32)

# Memory Management

This snippet provides two helper functions for managing and monitoring GPU memory when working with CuPy.

The `gpu_cleanup` function forces GPU memory cleanup by synchronizing the current CUDA stream and releasing all blocks from the CuPy default memory pool. If `verbose=True`, it will confirm whether cleanup completed successfully or print an error message if it fails.

The `gpu_memory_check` function reports current GPU memory usage. It queries the active CUDA device for free and total memory, calculates used memory, and prints the status in megabytes along with an optional label. It returns the amount of free memory in MB. If the check fails, it prints an error and returns zero.

A quick test at the end prints the installed CuPy version and attempts to display the GPU device’s free and total memory, allowing immediate confirmation that memory reporting works.


In [5]:
def gpu_cleanup(verbose=False):
    """Force GPU memory cleanup."""
    try:
        cp.cuda.get_current_stream().synchronize()
        cp.get_default_memory_pool().free_all_blocks()
        if verbose:
            print("GPU cleanup completed")
    except Exception as e:
        if verbose:
            print(f"GPU cleanup failed: {e}")

def gpu_memory_check(label=""):
    """Quick GPU memory status check using device memory info."""
    try:
        device = cp.cuda.Device()
        free_bytes, total_bytes = device.mem_info
        used_bytes = total_bytes - free_bytes
        
        used_mb = used_bytes / (1024**2)
        free_mb = free_bytes / (1024**2)
        total_mb = total_bytes / (1024**2)
        
        print(f"GPU Memory {label}: {used_mb:.1f}MB used, {free_mb:.1f}MB free ({total_mb:.1f}MB total)")
        return free_mb
    except Exception as e:
        print(f"GPU memory check failed: {e}")
        return 0

# Test the memory reporting
print(f"CuPy version: {cp.__version__}")
try:
    device = cp.cuda.Device()
    total, free = device.mem_info
    print(f"Device memory: {free/(1024**2):.1f}MB free / {total/(1024**2):.1f}MB total")
except Exception as e:
    print(f"Device memory check failed: {e}")



CuPy version: 13.5.1
Device memory: 16302.6MB free / 14865.0MB total


# Generate Data

The `VectorizedBinaryPolynomialFunction` class is the engine for GPU-accelerated synthetic data generation and coefficient recovery within APS experiments. It provides a suite of static methods that enable batch creation of experimental datasets with continuous and binary features, configurable functional forms (additive, multiplicative, or mixed), and flexible interaction structures.

The `generate_batch_mixed_data` method supports multiple experiments at once by grouping similar configurations, seeding the GPU random number generator for reproducibility, and generating feature–response pairs `(X, y_true, y_noisy)` directly on the GPU. Internally, `_group_similar_configs` clusters experiments based on function type, number of features, polynomial degree, and interaction types, ensuring that generation logic can be reused efficiently. For individual datasets, `_generate_single_experiment` samples continuous and binary features on the GPU, combines them, and produces outcomes using additive, multiplicative, or hybrid response models that incorporate both main effects and specified interactions. Noise can be injected directly on the GPU, with safeguards in place to keep outputs non-negative when required.

The class also defines helper routines for generating continuous and binary features, along with vectorized GPU implementations of additive, multiplicative, and mixed response functions. To support interpretability and validation, the method `get_true_marginal_coefficients_fixed` attempts to recover the underlying “true” marginal coefficients for each dataset. In additive models, this can often be derived analytically, while in multiplicative or mixed models the recovery is performed by fitting a polynomial model in GPU memory, with log-space adjustments applied when necessary.

Together, these methods provide a fast, scalable, and consistent framework for producing synthetic experiments that capture complex real-world feature–outcome relationships while still remaining tractable for surrogate modeling and coefficient comparison.


In [6]:
class VectorizedBinaryPolynomialFunction:
    """Vectorized polynomial function generator for batch experiment processing"""
    
# requires: import cupy as cp

    @staticmethod
    def generate_batch_mixed_data(experiment_configs, base_seed: int = 42):
        # set up fast CuPy allocators
        cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
        cp.cuda.set_pinned_memory_allocator(cp.cuda.PinnedMemoryPool().malloc)
        _ = cp.random.random((1,), dtype=cp.float32)  # warm up
    
        print(f"🔄 Generating data for {len(experiment_configs)} experiments on GPU (CuPy)…")
    
        batch_data = []
    
        # Group configs (unchanged)
        config_groups = VectorizedBinaryPolynomialFunction._group_similar_configs(experiment_configs)
    
        for group_idx, (_, configs) in enumerate(config_groups.items()):
            if group_idx % 5 == 0:
                print(f"  Processing group {group_idx+1}/{len(config_groups)}")
    
            for config in configs:
                seed = int(base_seed + int(config["experiment_id"]))
                cp.random.seed(seed)
    
                try:
                    # We convert outputs to CuPy so downstream stays on GPU.
                    X, y_true, y_noisy, feature_info = VectorizedBinaryPolynomialFunction._generate_single_experiment(config)
    
                    X = cp.asarray(X, dtype=cp.float32)
                    y_true = cp.asarray(y_true, dtype=cp.float32)
                    y_noisy = cp.asarray(y_noisy, dtype=cp.float32)
    
                    batch_data.append((X, y_true, y_noisy, feature_info, config))
    
                except Exception as e:
                    if group_idx % 10 == 0:
                        print(f"    Error in experiment {config['experiment_id']}: {str(e)[:50]}")
                    continue
    
        cp.cuda.Stream.null.synchronize()
        print(f"✅ Generated data for {len(batch_data)} valid experiments (GPU)")
        return batch_data

    
    @staticmethod
    def _group_similar_configs(configs):
        """Group configurations by similar parameters for batch processing"""
        groups = {}
        for config in configs:
            # Create grouping key based on data generation parameters
            key = (
                config['function_type'],
                config['n_continuous'], 
                config['n_binary'],
                config['degree'],
                tuple(sorted(config['interaction_types']))  # Sort for consistent grouping
            )
            
            if key not in groups:
                groups[key] = []
            groups[key].append(config)
        
        return groups
    
# requires: import cupy as cp

    @staticmethod
    def _generate_single_experiment(config):
        """Generate data for a single experiment on GPU (CuPy)."""
        n_samples        = int(config['n_samples'])
        n_continuous     = int(config['n_continuous'])
        n_binary         = int(config['n_binary'])
        degree           = int(config['degree'])  # kept for parity; not directly used here
        noise_level      = float(config['noise_level'])
        function_type    = config['function_type']
        binary_strength  = float(config['binary_effect_strength'])
        interaction_types= config['interaction_types']
    
        dt = cp.float32
    
        # --- Generate features on GPU ---
        if n_continuous > 0:
            if function_type == 'additive':
                X_cont = cp.random.uniform(-1.0, 1.0, size=(n_samples, n_continuous)).astype(dt)
            else:  # multiplicative or mixed
                X_cont = cp.random.uniform(0.1, 2.0, size=(n_samples, n_continuous)).astype(dt)
        else:
            X_cont = cp.empty((n_samples, 0), dtype=dt)
    
        if n_binary > 0:
            # p=0.4 Bernoulli
            X_bin = cp.random.binomial(1, 0.4, size=(n_samples, n_binary)).astype(dt)
        else:
            X_bin = cp.empty((n_samples, 0), dtype=dt)
    
        # --- Combine features ---
        if n_continuous > 0 and n_binary > 0:
            X = cp.concatenate([X_cont, X_bin], axis=1)
        elif n_continuous > 0:
            X = X_cont
        elif n_binary > 0:
            X = X_bin
        else:
            raise ValueError("Must have at least one feature")
    
        # --- Response generation on GPU ---
        if function_type == 'additive':
            y = cp.zeros(n_samples, dtype=dt)
    
            # continuous main effects
            if n_continuous > 0:
                y += cp.sum(2.0 * X_cont**2 + 1.0 * X_cont, axis=1)
    
            # binary main effects
            if n_binary > 0:
                y += (binary_strength * 3.0) * cp.sum(X_bin, axis=1)
    
            # interactions
            if 'continuous' in interaction_types and n_continuous > 1:
                y += 1.5 * X_cont[:, 0] * X_cont[:, 1]
    
            if 'binary' in interaction_types and n_binary > 1:
                y += (binary_strength * 2.0) * X_bin[:, 0] * X_bin[:, 1]
    
            if 'mixed' in interaction_types and n_continuous > 0 and n_binary > 0:
                y += (binary_strength * 1.8) * X_cont[:, 0] * X_bin[:, 0]
    
            y += 5.0
    
        elif function_type == 'multiplicative':
            y = cp.ones(n_samples, dtype=dt) * 2.0
    
            if n_continuous > 0:
                factors = 1.0 + X_cont + 0.3 * (X_cont**2)
                y *= cp.prod(factors, axis=1)
    
            if n_binary > 0:
                bf = 1.0 + (binary_strength * 0.8) * X_bin
                y *= cp.prod(bf, axis=1)
    
            if 'continuous' in interaction_types and n_continuous > 1:
                y *= (1.0 + 0.2 * X_cont[:, 0] * X_cont[:, 1])
    
            if 'binary' in interaction_types and n_binary > 1:
                y *= (1.0 + (binary_strength * 0.3) * X_bin[:, 0] * X_bin[:, 1])
    
            if 'mixed' in interaction_types and n_continuous > 0 and n_binary > 0:
                y *= (1.0 + (binary_strength * 0.25) * X_cont[:, 0] * X_bin[:, 0])
    
            y *= 1.5
    
        else:  # 'mixed'
            # additive branch
            y_add = cp.zeros(n_samples, dtype=dt)
            if n_continuous > 0:
                y_add += cp.sum(2.0 * X_cont**2 + 1.0 * X_cont, axis=1)
            if n_binary > 0:
                y_add += (binary_strength * 3.0) * cp.sum(X_bin, axis=1)
            if 'continuous' in interaction_types and n_continuous > 1:
                y_add += 1.5 * X_cont[:, 0] * X_cont[:, 1]
            if 'binary' in interaction_types and n_binary > 1:
                y_add += (binary_strength * 2.0) * X_bin[:, 0] * X_bin[:, 1]
            if 'mixed' in interaction_types and n_continuous > 0 and n_binary > 0:
                y_add += (binary_strength * 1.8) * X_cont[:, 0] * X_bin[:, 0]
            y_add += 5.0
    
            # multiplicative branch
            y_mul = cp.ones(n_samples, dtype=dt) * 2.0
            if n_continuous > 0:
                factors = 1.0 + X_cont + 0.3 * (X_cont**2)
                y_mul *= cp.prod(factors, axis=1)
            if n_binary > 0:
                bf = 1.0 + (binary_strength * 0.8) * X_bin
                y_mul *= cp.prod(bf, axis=1)
            if 'continuous' in interaction_types and n_continuous > 1:
                y_mul *= (1.0 + 0.2 * X_cont[:, 0] * X_cont[:, 1])
            if 'binary' in interaction_types and n_binary > 1:
                y_mul *= (1.0 + (binary_strength * 0.3) * X_bin[:, 0] * X_bin[:, 1])
            if 'mixed' in interaction_types and n_continuous > 0 and n_binary > 0:
                y_mul *= (1.0 + (binary_strength * 0.25) * X_cont[:, 0] * X_bin[:, 0])
            y_mul *= 1.5
    
            # normalize & mix
            add_mu, add_sd = cp.mean(y_add), cp.std(y_add) + 1e-8
            mul_mu, mul_sd = cp.mean(y_mul), cp.std(y_mul) + 1e-8
            y_add_n = (y_add - add_mu) / add_sd
            y_mul_n = (y_mul - mul_mu) / mul_sd
            additive_weight = float(config.get('additive_weight', 0.5))
            y = additive_weight * y_add_n + (1.0 - additive_weight) * y_mul_n
            y = y * 3.0 + 10.0
    
        y_true = y.copy()
    
        # --- Noise on GPU ---
        if noise_level > 0.0:
            sigma = float(noise_level) * float(cp.std(y))
            noise = cp.random.normal(0.0, sigma, size=n_samples).astype(dt)
            y_noisy = y + noise
        else:
            y_noisy = y.copy()
    
        # --- Ensure positivity if needed ---
        ymin = float(cp.min(y_noisy))
        if ymin <= 0.0:
            shift = abs(ymin) + 0.1
            y_noisy = y_noisy + cp.asarray(shift, dtype=dt)
            y_true  = y_true  + cp.asarray(shift, dtype=dt)
    
        feature_info = {
            'n_continuous': n_continuous,
            'n_binary': n_binary,
            'continuous_indices': list(range(n_continuous)),
            'binary_indices': list(range(n_continuous, n_continuous + n_binary)),
            'interaction_types': interaction_types,
            'binary_effect_strength': binary_strength,
        }
    
        return X, y_true, y_noisy, feature_info

    
    @staticmethod
    def _generate_continuous_features(n_samples, n_continuous, function_type, dtype=cp.float32, rng=None):
        """Vectorized continuous feature generation on GPU (CuPy)."""
        if n_continuous == 0:
            return cp.empty((n_samples, 0), dtype=dtype)
        rng = rng or cp.random
        if function_type == 'additive':
            return rng.uniform(-1.0, 1.0, size=(n_samples, n_continuous)).astype(dtype)
        else:  # multiplicative or mixed
            return rng.uniform(0.1, 2.0, size=(n_samples, n_continuous)).astype(dtype)
        
    @staticmethod
    def _generate_binary_features(n_samples, n_binary, dtype=cp.int8, rng=None):
        """Vectorized binary feature generation on GPU (CuPy)."""
        if n_binary == 0:
            return cp.empty((n_samples, 0), dtype=dtype)
        rng = rng or cp.random
        return rng.binomial(1, 0.4, size=(n_samples, n_binary)).astype(dtype)
    
    @staticmethod
    def _compute_additive_response(X_cont, X_bin, binary_strength, interaction_types, dtype=cp.float32):
        """Vectorized additive response on GPU (CuPy)."""
        n_samples = X_cont.shape[0] if X_cont.size else X_bin.shape[0]
        y = cp.zeros(n_samples, dtype=dtype)
    
        n_cont = X_cont.shape[1]
        n_bin  = X_bin.shape[1]
    
        # continuous main effects: sum(2*x^2 + 1*x)
        if n_cont > 0:
            y += cp.sum(2.0 * (X_cont.astype(dtype, copy=False) ** 2) +
                        1.0 * X_cont.astype(dtype, copy=False), axis=1)
    
        # binary main effects: sum(binary_strength * 3 * x)
        if n_bin > 0:
            xb = X_bin.astype(dtype, copy=False)
            y += (binary_strength * 3.0) * cp.sum(xb, axis=1)
    
        # interactions
        if 'continuous' in interaction_types and n_cont > 1:
            y += 1.5 * X_cont[:, 0].astype(dtype, copy=False) * X_cont[:, 1].astype(dtype, copy=False)
    
        if 'binary' in interaction_types and n_bin > 1:
            xb = X_bin.astype(dtype, copy=False)
            y += (binary_strength * 2.0) * xb[:, 0] * xb[:, 1]
    
        if 'mixed' in interaction_types and n_cont > 0 and n_bin > 0:
            xb0 = X_bin[:, 0].astype(dtype, copy=False)
            y += (binary_strength * 1.8) * X_cont[:, 0].astype(dtype, copy=False) * xb0
    
        # base level
        y += 5.0
        return y

    @staticmethod
    def _compute_multiplicative_response(X_cont, X_bin, binary_strength, interaction_types, dtype=cp.float32):
        """Vectorized multiplicative response on GPU (CuPy)."""
        n_samples = X_cont.shape[0]
        n_cont = X_cont.shape[1]
        n_bin  = X_bin.shape[1]
    
        Xc = X_cont.astype(dtype, copy=False)
        Xb = X_bin.astype(dtype, copy=False)
    
        y = cp.ones(n_samples, dtype=dtype) * 2.0
    
        # continuous multiplicative factors
        if n_cont > 0:
            factors = 1.0 + Xc + 0.3 * (Xc ** 2)
            y *= cp.prod(factors, axis=1)
    
        # binary multiplicative factors
        if n_bin > 0:
            bf = 1.0 + (binary_strength * 0.8) * Xb
            y *= cp.prod(bf, axis=1)
    
        # interactions
        if 'continuous' in interaction_types and n_cont > 1:
            y *= (1.0 + 0.2 * Xc[:, 0] * Xc[:, 1])
    
        if 'binary' in interaction_types and n_bin > 1:
            y *= (1.0 + (binary_strength * 0.3) * Xb[:, 0] * Xb[:, 1])
    
        if 'mixed' in interaction_types and n_cont > 0 and n_bin > 0:
            y *= (1.0 + (binary_strength * 0.25) * Xc[:, 0] * Xb[:, 0])
    
        y *= 1.5
        return y
    
    @staticmethod
    def _compute_mixed_response(
        X_cont,
        X_bin,
        binary_strength,
        interaction_types,
        additive_weight: float = 0.5,
        dtype=cp.float32,
    ):
        """Vectorized mixed response on GPU (CuPy)."""
        # compute branches on GPU
        y_add = VectorizedBinaryPolynomialFunction._compute_additive_response(
            X_cont, X_bin, binary_strength, interaction_types, dtype=dtype
        ).astype(dtype, copy=False)
    
        y_mul = VectorizedBinaryPolynomialFunction._compute_multiplicative_response(
            X_cont, X_bin, binary_strength, interaction_types, dtype=dtype
        ).astype(dtype, copy=False)
    
        # normalize each branch
        eps    = cp.asarray(1e-8, dtype=dtype)
        add_mu = cp.mean(y_add)
        add_sd = cp.std(y_add) + eps
        mul_mu = cp.mean(y_mul)
        mul_sd = cp.std(y_mul) + eps
    
        y_add_n = (y_add - add_mu) / add_sd
        y_mul_n = (y_mul - mul_mu) / mul_sd
    
        # mix and shift
        aw = float(additive_weight)
        y_mixed = aw * y_add_n + (1.0 - aw) * y_mul_n
        return y_mixed * 3.0 + 10.0

    @staticmethod
    def get_true_marginal_coefficients_fixed(
        n_continuous, n_binary, binary_strength, function_type,
        X_sample=None, y_sample=None, additive_weight=0.5, dtype=cp.float32
    ):
        """
        GPU version: estimate 'true' marginal coefficients by fitting a degree-2
        polynomial model on GPU. For multiplicative: fit in log-space.
        Returns a dict with intercept and main-effect (linear/quadratic) terms only.
        """
        true_coeffs = {}
        p_cont, p_bin = int(n_continuous), int(n_binary)
        p = p_cont + p_bin
    
        # ----- Case 1: additive -> coefficients are known from construction -----
        if function_type == 'additive':
            for i in range(p_cont):
                true_coeffs[f'x{i}_linear'] = 1.0
                true_coeffs[f'x{i}_quadratic'] = 2.0
            for j in range(p_bin):
                true_coeffs[f'x{p_cont + j}_binary'] = float(binary_strength) * 3.0
            true_coeffs['intercept'] = 5.0
            return true_coeffs
    
        # If no sample provided, fall back to the original approximations
        if X_sample is None or y_sample is None:
            if function_type == 'multiplicative':
                true_coeffs['intercept'] = math.log(2.0 * 1.5)
                for i in range(p_cont):
                    true_coeffs[f'x{i}_linear'] = 1.0
                    true_coeffs[f'x{i}_quadratic'] = 0.3
                for j in range(p_bin):
                    true_coeffs[f'x{p_cont + j}_binary'] = float(binary_strength) * 0.8
                return true_coeffs
            else:  # mixed fallback
                for i in range(p_cont):
                    true_coeffs[f'x{i}_linear'] = 1.0
                    true_coeffs[f'x{i}_quadratic'] = 0.5
                for j in range(p_bin):
                    true_coeffs[f'x{p_cont + j}_binary'] = float(binary_strength) * 2.0
                true_coeffs['intercept'] = 10.0
                return true_coeffs
    
        # ----- GPU path with samples -----
        X = cp.asarray(X_sample, dtype=dtype)
        y = cp.asarray(y_sample, dtype=dtype)
    
        # Build degree-2 polynomial design on GPU:
        # Design columns = [1, x_i (all), x_i^2 (continuous only), x_i*x_j (all i<j)]
        cols = [cp.ones((X.shape[0], 1), dtype=dtype)]  # intercept
    
        # linear terms (all variables)
        cols.append(X)
    
        # quadratic terms for continuous only
        if p_cont > 0:
            cols.append(X[:, :p_cont] ** 2)
    
        # pairwise interactions (all i<j)
        if p > 1:
            inter_cols = []
            for i in range(p):
                Xi = X[:, i:i+1]
                for j in range(i+1, p):
                    inter_cols.append(Xi * X[:, j:j+1])
            if inter_cols:
                cols.append(cp.concatenate(inter_cols, axis=1))
    
        X_poly = cp.concatenate(cols, axis=1)
    
        # Target vector: log-space for multiplicative; original y for mixed
        if function_type == 'multiplicative':
            ymin = float(cp.min(y))
            y_pos = y - ymin + 1.0 if ymin <= 0.0 else y
            t = cp.log(y_pos)
        else:  # 'mixed'
            t = y
    
        # Solve least squares on GPU
        # beta shape: (n_features,)
        beta, *_ = cp.linalg.lstsq(X_poly, t, rcond=None)
    
        # Map back to main effects
        # index layout:
        #   0                      -> intercept
        #   1 .. p                 -> linear terms x0..x{p-1}
        #   p+1 .. p+p_cont        -> quadratic terms (continuous only): x0^2..x{p_cont-1}^2
        #   remaining              -> interactions (ignored for marginal coeffs)
        idx = 0
        intercept = float(beta[idx].item()); idx += 1
    
        # linear
        for i in range(p):
            true_coeffs[f'x{i}_linear' if i < p_cont else f'x{i}_binary'] = float(beta[idx + i].item())
        idx += p
    
        # quadratic (continuous only)
        for i in range(p_cont):
            true_coeffs[f'x{i}_quadratic'] = float(beta[idx + i].item())
        idx += p_cont
    
        true_coeffs['intercept'] = intercept
        return true_coeffs


# Evaluate Marginal Coefficient Recovery

This function, **`evaluate_marginal_coefficient_recovery_fixed`**, measures how well the fitted AMS/APS model recovers the true main-effect coefficients on the GPU. It begins by checking if a model fit exists; if not, it returns default values. The training data is then moved to CuPy, the number of continuous and binary predictors is inferred, and a **main-effects-only polynomial design matrix** is constructed up to degree `D = ams_model.max_degree`. Continuous variables contribute powers \(1..D\) (linear, quadratic, cubic, etc.), while binary variables contribute only a single linear column (since \(b^k = b\), preventing collinearity). Depending on the selected method, the target vector is transformed—**MAPS** fits in log-space with a positivity shift, while **APS** uses the original space—and a least-squares solution is solved directly on the GPU. The recovered coefficients are mapped back to interpretable names (e.g., `x0_linear`, `x0_quadratic`, `x3_binary`, plus `intercept`) and compared against `true_coeffs`. Only shared, non-NaN terms are included in the comparison. The function then computes **sign accuracy**, **absolute-magnitude correlation** (with a stable fallback for single coefficients), and **RMSE** between recovered and true values. Finally, it returns these metrics along with the number of coefficients compared, providing a GPU-accelerated checkpoint for coefficient direction, magnitude, and error—fully aligned with APS/MAPS design choices.


In [7]:
def evaluate_marginal_coefficient_recovery_fixed(
    ams_model,
    X_train,
    y_train,
    true_coeffs,
    n_continuous,
    n_binary,
    dtype=cp.float32,
):
    """
    GPU version (CuPy):
      - Main-effects polynomial design up to degree D = ams_model.max_degree
      - Continuous vars: powers 1..D
      - Binary vars: linear only (b^k == b), to avoid collinearity
      - MAPS => fit in log-space with positive shift; APS => original space
      - Metrics computed on GPU
    """
    if getattr(ams_model, "final_model", None) is None:
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': float('inf'),
            'n_compared': 0
        }

    # move data to GPU
    X = cp.asarray(X_train, dtype=dtype)
    y = cp.asarray(y_train, dtype=dtype)

    p_cont = int(n_continuous)
    p_bin  = int(n_binary)
    p      = p_cont + p_bin
    D      = int(getattr(ams_model, "max_degree", 3))

    # ---- build main-effects polynomial design up to degree D ----
    # columns: [1, (for each feature i) x_i^1, x_i^2..x_i^D (continuous only)]
    cols = [cp.ones((X.shape[0], 1), dtype=dtype)]  # intercept

    # continuous variables: powers 1..D
    for i in range(p_cont):
        Xi = X[:, i:i+1]
        # stack powers (1..D)
        powers = [Xi] + [Xi ** k for k in range(2, D + 1)]
        cols.append(cp.concatenate(powers, axis=1))

    # binary variables: linear only to avoid duplicate cols
    for j in range(p_cont, p):
        cols.append(X[:, j:j+1])

    X_poly = cp.concatenate(cols, axis=1)

    # target: MAPS => log-space with shift; APS => original
    if getattr(ams_model, 'selected_method', 'APS') == 'MAPS':
        ymin = float(cp.min(y))
        y_pos = y - ymin + 1.0 if ymin <= 0.0 else y
        t = cp.log(y_pos)
    else:
        t = y

    # ---- solve least squares on GPU ----
    beta, *_ = cp.linalg.lstsq(X_poly, t, rcond=None)

    # ---- map coefficients back to names consistent with your dict ----
    coeffs_rec = {}
    idx = 0

    # intercept
    coeffs_rec['intercept'] = float(beta[idx].item()); idx += 1

    # continuous vars: degrees 1..D
    for i in range(p_cont):
        for k in range(1, D + 1):
            b = float(beta[idx].item()); idx += 1
            if   k == 1: key = f'x{i}_linear'
            elif k == 2: key = f'x{i}_quadratic'
            elif k == 3: key = f'x{i}_cubic'
            else:        key = f'x{i}_power{k}'
            coeffs_rec[key] = b

    # binary vars: linear only
    for j in range(p_cont, p):
        b = float(beta[idx].item()); idx += 1
        coeffs_rec[f'x{j}_binary'] = b  # (linear effect for binary)

    # ---- compare to provided true coefficients (GPU) ----
    common = [k for k in coeffs_rec if (k in true_coeffs) and (true_coeffs[k] is not None)]
    if not common:
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': float('inf'),
            'n_compared': 0
        }

    tv = cp.asarray([true_coeffs[k] for k in common], dtype=cp.float64)
    fv = cp.asarray([coeffs_rec[k]  for k in common], dtype=cp.float64)

    mask = ~cp.isnan(tv)
    if not bool(mask.any()):
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': float('inf'),
            'n_compared': 0
        }
    tv = tv[mask]; fv = fv[mask]
    n = int(tv.size)
    if n == 0:
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': float('inf'),
            'n_compared': 0
        }

    # sign accuracy
    sign_acc = float(cp.mean(cp.sign(tv) == cp.sign(fv)).item())

    # magnitude correlation (abs values); special-case n==1
    if n > 1:
        r = cp.corrcoef(cp.abs(tv), cp.abs(fv))[0, 1]
        mag_corr = float(cp.nan_to_num(r, nan=0.0).item())
    else:
        rel_err = float(cp.abs(tv[0] - fv[0]) / (cp.abs(tv[0]) + 1e-10))
        mag_corr = max(0.0, 1.0 - rel_err)

    # RMSE
    rmse = float(cp.sqrt(cp.mean((tv - fv) ** 2)).item())

    return {
        'sign_accuracy': sign_acc,
        'magnitude_correlation': mag_corr,
        'coefficient_rmse': rmse,
        'n_compared': n
    }


# Quantile Remapper

The **`QuantileRemapper`** class provides a GPU-accelerated way to remap feature values based on their quantile distributions, improving interpretability of MAPS coefficients. It works entirely with **CuPy arrays** to leverage GPU speed. When `fit` is called, the class processes each variable, recording its original range and constructing a quantile mapping: sorted source values are paired with evenly spaced target values over the variable’s range. If a feature is constant, it simply maps to zeros. The `transform` method applies this mapping to new data by using `cp.interp`, effectively normalizing each variable into a quantile-based scale while preserving its distribution. Conversely, `inverse_transform` reverses this mapping, restoring values back to their original scale using the stored quantile functions. Internal safeguards ensure that input arrays are CuPy-based, and the class tracks whether it has been fitted before use. This remapping provides a consistent framework for comparing coefficients across heterogeneous variables, particularly important when interpreting polynomial effects in MAPS models.


In [8]:
import cupy as cp

class QuantileRemapper:
    """Quantile-based variable remapping for MAPS coefficient interpretability (GPU/CuPy)"""

    def __init__(self):
        self.quantile_functions = {}
        self.original_ranges = {}
        self.is_fitted = False

    def _ensure_gpu(self, X):
        if not isinstance(X, cp.ndarray):
            raise TypeError("QuantileRemapper expects a CuPy array (cupy.ndarray).")

    def fit(self, X: cp.ndarray):
        """Fit quantile mapping functions for each variable on GPU"""
        self._ensure_gpu(X)
        self.quantile_functions = {}
        self.original_ranges = {}

        n_features = X.shape[1]
        for i in range(n_features):
            var_data = cp.ascontiguousarray(X[:, i])
            vmin = var_data.min()
            vmax = var_data.max()
            original_range = vmax - vmin
            self.original_ranges[i] = original_range

            sorted_values = cp.sort(var_data)
            n_unique = cp.unique(sorted_values).size

            if n_unique > 1:
                # Map quantiles to (0, original_range]
                target_values = cp.linspace(
                    0.01,
                    (original_range + cp.asarray(0.01, dtype=sorted_values.dtype)),
                    sorted_values.size,
                    dtype=sorted_values.dtype,
                )
                self.quantile_functions[i] = {
                    "source_values": sorted_values,
                    "target_values": target_values,
                }
            else:
                self.quantile_functions[i] = {
                    "source_values": sorted_values,
                    "target_values": cp.zeros_like(sorted_values),
                }

        self.is_fitted = True
        return self

    def transform(self, X: cp.ndarray) -> cp.ndarray:
        """Apply quantile remapping to variables on GPU"""
        if not self.is_fitted:
            raise ValueError("Must fit before transform")
        self._ensure_gpu(X)

        X_remapped = cp.zeros_like(X)
        n_features = X.shape[1]
        for i in range(n_features):
            var_data = cp.ascontiguousarray(X[:, i])
            mapping = self.quantile_functions[i]
            # cp.interp operates on 1D arrays (ascending x assumed)
            remapped = cp.interp(
                var_data.astype(mapping["source_values"].dtype, copy=False),
                mapping["source_values"],
                mapping["target_values"],
            )
            X_remapped[:, i] = remapped.astype(X.dtype, copy=False)

        return X_remapped

    def inverse_transform(self, X_remapped: cp.ndarray) -> cp.ndarray:
        """Inverse quantile remapping back to original scale on GPU"""
        if not self.is_fitted:
            raise ValueError("Must fit before inverse_transform")
        self._ensure_gpu(X_remapped)

        X_original = cp.zeros_like(X_remapped)
        n_features = X_remapped.shape[1]
        for i in range(n_features):
            var_data = cp.ascontiguousarray(X_remapped[:, i])
            mapping = self.quantile_functions[i]
            restored = cp.interp(
                var_data.astype(mapping["target_values"].dtype, copy=False),
                mapping["target_values"],
                mapping["source_values"],
            )
            X_original[:, i] = restored.astype(X_remapped.dtype, copy=False)

        return X_original


# Torch Linear Regression

The **`TorchLinearRegression`** class implements a GPU/CPU-flexible linear regression solver in PyTorch with optional CuPy interoperability for seamless use in GPU-based workflows. The constructor allows configuration of whether to fit an intercept, target device (`cpu` or `cuda`), numeric precision (`dtype`), and solver strategy (`lstsq` for robustness or `normal` equations for speed). Internally, helper methods safely convert between CuPy and Torch tensors via DLPack while preserving data ownership. The `fit` method constructs the design matrix (adding a bias column if requested), solves for coefficients using the specified method, and stores both Torch tensors and CuPy-accessible versions of weights and intercepts. The `predict` method generates outputs from new data, while `score` computes the coefficient of determination (R²) to evaluate model performance. A specialized `_fast_cv_score` method performs k-fold cross-validation entirely in Torch to avoid DLPack hazards, ensuring safe and efficient GPU computations. Overall, this design provides a flexible and high-performance linear regression class, bridging PyTorch’s optimization routines with CuPy integration for workflows that demand both speed and compatibility with GPU array libraries.


In [9]:
class TorchLinearRegression:
    def __init__(
        self,
        fit_intercept: bool = True,
        device: Optional[str] = None,   # None -> 'cuda' if available else 'cpu'
        dtype: torch.dtype = torch.float32,
        solver: str = "lstsq",          # 'lstsq' (robust) or 'normal'
    ):
        self.fit_intercept = fit_intercept
        self.device = device
        self.dtype = dtype
        self.solver = solver

        # learned params exposed as CuPy for your workflow
        self.coef_: Optional["cp.ndarray"] = None          # (n_features,) or (n_targets, n_features)
        self.intercept_: Optional["cp.ndarray"] = None     # () or (n_targets,)

        # internal torch copies
        self._coef_t: Optional[torch.Tensor] = None        # (n_targets, n_features)
        self._intercept_t: Optional[torch.Tensor] = None   # (n_targets,)

    # -------- helpers --------
    def _infer_device(self) -> str:
        if self.device is not None:
            return self.device
        return "cuda" if torch.cuda.is_available() else "cpu"

    @staticmethod
    def _is_cupy(x: Any) -> bool:
        return _HAS_CUPY and (type(x).__module__.split(".")[0] == "cupy")

    @staticmethod
    def _torch_to_cupy(t: torch.Tensor) -> "cp.ndarray":
        if not _HAS_CUPY:
            raise RuntimeError("CuPy is required for cp.ndarray outputs.")
        if t.device.type != "cuda":
            if not torch.cuda.is_available():
                raise RuntimeError("CUDA not available for cp.ndarray output.")
            t = t.to("cuda")
        torch.cuda.synchronize()
        from_dl = getattr(cp, "from_dlpack", None) or getattr(cp, "fromDlpack", None)
        return from_dl(torch_dlpack.to_dlpack(t.detach()))

    @staticmethod
    def _cupy_to_torch_owning(x_cp_src: "cp.ndarray", device: str, dtype: torch.dtype) -> torch.Tensor:
        # make a private, contiguous GPU copy so the original CuPy array remains safe after DLPack handoff
        x_cp = cp.ascontiguousarray(x_cp_src)
        cp.cuda.get_current_stream().synchronize()
        to_dl = getattr(x_cp, "toDlpack", None) or getattr(x_cp, "to_dlpack", None)
        t = torch_dlpack.from_dlpack(to_dl())
        if device:
            t = t.to(device)
        if dtype:
            t = t.to(dtype)
        return t

    def _to_tensor(self, x: _Array, device: str, dtype: torch.dtype) -> torch.Tensor:
        if torch.is_tensor(x):
            return x.to(device=device, dtype=dtype)
        if self._is_cupy(x):
            return self._cupy_to_torch_owning(x, device, dtype)
        raise TypeError(f"Unsupported array type: {type(x)}")

    def _from_tensor(self, t: torch.Tensor) -> "cp.ndarray":
        return self._torch_to_cupy(t)

    # -------- core API --------
    def get_params(self, deep: bool = True) -> Dict[str, Any]:
        return {
            "fit_intercept": self.fit_intercept,
            "device": self.device,
            "dtype": self.dtype,
            "solver": self.solver,
        }

    def set_params(self, **params) -> "TorchLinearRegression":
        for k, v in params.items():
            setattr(self, k, v)
        return self

    def fit(self, X: _Array, y: _Array) -> "TorchLinearRegression":
        dev = self._infer_device()
        X_t = self._to_tensor(X, dev, self.dtype)
        y_t = self._to_tensor(y, dev, self.dtype)

        if y_t.ndim == 1:
            y_t = y_t.unsqueeze(1)

        if self.fit_intercept:
            ones = torch.ones((X_t.shape[0], 1), device=dev, dtype=self.dtype)
            X_aug = torch.cat([X_t, ones], dim=1)
        else:
            X_aug = X_t

        if self.solver == "normal":
            XtX = X_aug.T @ X_aug
            XtX = XtX + 1e-8 * torch.eye(XtX.shape[0], device=dev, dtype=self.dtype)
            Xty = X_aug.T @ y_t
            beta = torch.linalg.solve(XtX, Xty)
        else:
            beta = torch.linalg.lstsq(X_aug, y_t).solution

        if self.fit_intercept:
            w = beta[:-1, :]       # (n_features, n_targets)
            b = beta[-1, :]        # (n_targets,)
        else:
            w = beta
            b = torch.zeros((y_t.shape[1],), device=dev, dtype=self.dtype)

        self._coef_t = w.T.contiguous()      # (n_targets, n_features)
        self._intercept_t = b.contiguous()   # (n_targets,)

        # expose CuPy params
        if y_t.shape[1] == 1:
            self.coef_ = self._from_tensor(self._coef_t)[0]            # (n_features,)
            self.intercept_ = self._from_tensor(self._intercept_t)[0]  # 0-d cp array
        else:
            self.coef_ = self._from_tensor(self._coef_t)               # (n_targets, n_features)
            self.intercept_ = self._from_tensor(self._intercept_t)     # (n_targets,)

        return self

    def predict(self, X: _Array) -> "cp.ndarray":
        if self._coef_t is None or self._intercept_t is None:
            raise RuntimeError("Model is not fitted.")
        dev = self._coef_t.device.type
        X_t = self._to_tensor(X, dev, self._coef_t.dtype)
        y = X_t @ self._coef_t.T
        y = y + (self._intercept_t if self._intercept_t.ndim != 0 else self._intercept_t)
        return self._from_tensor(y.squeeze(-1))

    def score(self, X: _Array, y: _Array) -> float:
        if self._coef_t is None:
            raise RuntimeError("Model is not fitted.")
        dev = self._infer_device()
        X_t = self._to_tensor(X, dev, self._coef_t.dtype)
        y_t = self._to_tensor(y, dev, self._coef_t.dtype)
        if y_t.ndim == 1:
            y_t = y_t.unsqueeze(1)

        y_hat = X_t @ self._coef_t.T
        y_hat = y_hat + (self._intercept_t if self._intercept_t.ndim != 0 else self._intercept_t)

        ss_res = torch.sum((y_t - y_hat) ** 2, dim=0)
        y_mean = torch.mean(y_t, dim=0, keepdim=True)
        ss_tot = torch.sum((y_t - y_mean) ** 2, dim=0).clamp_min(1e-12)
        r2 = 1.0 - (ss_res / ss_tot)
        return float(torch.mean(r2).item())

    # -------- CV stays in Torch to avoid DLPack hazards --------
        def _cv_r2_gpu(self, X_cp, y_cp, k=3):
            return _fast_cv_score(X_cp, y_cp, k)

# Additive Polynomial Selection

The **`VectorizedAPS`** class implements a GPU-accelerated version of *Additive Polynomial Selection (APS)* for nonlinear feature discovery and regression modeling. It combines **CuPy** for array-level GPU operations with **TorchLinearRegression** for efficient linear regression fits, making it well-suited for large-scale, high-dimensional datasets.

During the `fit` process, input data is first moved to the GPU and then expanded into polynomial features up to the specified degree. Candidate features are filtered according to interaction order, allowing restrictions on the number of variables included in each monomial. A greedy, vectorized forward-selection algorithm is then applied, evaluating candidates in parallel with GPU-backed cross-validation R² scores to identify the most informative polynomial terms. The final model is trained using **TorchLinearRegression** on the selected features, with a fallback to raw inputs if no candidates meet the tolerance threshold.

The `predict` method ensures consistency by regenerating polynomial features at inference and applying the same selected indices. Helper methods provide additional functionality such as filtering by interaction complexity, efficient candidate evaluation, and fast GPU-based cross-validation that avoids costly CPU–GPU transfers. A static method `_gpu_poly_features` constructs polynomial features and generates human-readable feature names directly in GPU memory, maintaining speed and interpretability.

Overall, the design of **`VectorizedAPS`** balances interpretability, efficiency, and scalability. It enables effective feature engineering and polynomial model selection in GPU workflows, making it a powerful tool for uncovering nonlinear relationships in high-dimensional data.


In [10]:
class VectorizedAPS:
    """Vectorized Additive Polynomial Selection (GPU/CuPy + Torch OLS)"""

    def __init__(self, max_degree=3, max_interaction_order=2, tolerance=1e-4, verbose=False):
        self.max_degree = int(max_degree)
        self.max_interaction_order = int(max_interaction_order)
        self.tolerance = float(tolerance)
        self.verbose = verbose
        self.selected_features = None           # indices within candidates
        self.final_model = None                 # TorchLinearRegression
        self._feature_names_all = None          # all poly feature names
        self._candidate_indices = None          # filtered indices (interaction <= K)

    # ---------- public API ----------
    def fit(self, X, y):
        # move to GPU 
        X_cp = cp.asarray(X)
        y_cp = cp.asarray(y).reshape(-1)

        # build polynomial features on GPU
        X_poly, feat_names = self._gpu_poly_features(X_cp, self.max_degree, include_bias=False)
        self._feature_names_all = feat_names

        # filter by max interaction order
        candidate_indices = self._filter_by_interaction_order(feat_names)
        if len(candidate_indices) == 0:
            candidate_indices = list(range(min(X_cp.shape[1], X_poly.shape[1])))

        X_cand = X_poly[:, candidate_indices]
        self._candidate_indices = candidate_indices

        # forward selection (GPU-backed CV)
        self.selected_features = self._vectorized_forward_selection(X_cand, y_cp)

        # fit final model on selected poly features (or fallback to raw X)
        from math import inf
        if self.selected_features and len(self.selected_features) > 0:
            X_final = X_cand[:, self.selected_features]
        else:
            # fallback — use raw X if nothing selected
            X_final = X_cp

        lr = TorchLinearRegression(fit_intercept=True, device="cuda")
        lr.fit(X_final, y_cp)
        self.final_model = lr

        # store absolute poly indices for prediction-time selection
        if self.selected_features and len(self.selected_features) > 0:
            abs_idxs = [candidate_indices[i] for i in self.selected_features]
        else:
            abs_idxs = []
        # piggyback “selected_features” like your original did
        self.final_model.selected_features = abs_idxs
        return self

    def predict(self, X):
        if self.final_model is None:
            return cp.zeros(len(X), dtype=cp.float32)   # or cp.float64 if you prefer

        X_cp = cp.asarray(X)

        # rebuild poly features and select the same columns
        if hasattr(self.final_model, "selected_features") and len(self.final_model.selected_features) > 0:
            X_poly, _ = self._gpu_poly_features(X_cp, self.max_degree, include_bias=False)
            abs_idxs = self.final_model.selected_features
            if len(abs_idxs) > 0 and max(abs_idxs) < X_poly.shape[1]:
                X_selected = X_poly[:, abs_idxs]
                return self.final_model.predict(X_selected)  # returns NumPy
        # fallback: predict on raw X
        return self.final_model.predict(X_cp)

    # ---------- helpers ----------
    def _filter_by_interaction_order(self, feature_names):
        """Keep features whose monomial uses ≤ max_interaction_order unique variables."""
        valid = []
        for i, name in enumerate(feature_names):
            # names look like: 'x0', 'x1^2', 'x0 x1', 'x0^2 x2', etc.
            var_matches = re.findall(r'x(\d+)', name)
            if len(set(var_matches)) <= self.max_interaction_order:
                valid.append(i)
        return valid

    def _vectorized_forward_selection(self, X_cand, y_cp):
        return _vectorized_forward_selection(X_cand, y_cp, self.tolerance, self.verbose)

    def _batch_evaluate_candidates(self, X_cand, y_cp, selected, remaining):
        """Evaluate a batch of candidates by k-fold R², stays on GPU for data/fit."""
        scores = []
        X_sel = X_cand[:, selected] if selected else None

        for cand in remaining:
            # assemble [selected | candidate]
            X_test = X_cand[:, [cand]] if X_sel is None else cp.concatenate([X_sel, X_cand[:, [cand]]], axis=1)
            score = self._fast_cv_score(X_test, y_cp, cv_folds=3)
            scores.append(score)

        return cp.asarray(scores, dtype=cp.float32)

    def _fast_cv_score(self, X_cp, y_cp, cv_folds=3):
        return _fast_cv_score(X_cp, y_cp, cv_folds)

    # ---------- GPU polynomial features ----------
    @staticmethod
    def _gpu_poly_features(X_cp: cp.ndarray, degree: int, include_bias: bool = False):
        return gpu_poly_features(X_cp, degree, include_bias)



# Multiplicative Polynomial Selection

The **`VectorizedMAPS`** class implements a GPU-accelerated approach to *Multiplicative Polynomial Selection (MAPS)* for modeling nonlinear relationships in data. Its workflow emphasizes interpretability and stability by combining quantile remapping, log-space transformations, and forward feature selection. In the `fit` stage, input data is transferred to GPU memory using CuPy, and the target variable is shifted into the positive domain (if needed) to allow log transformation. Explanatory variables are quantile-remapped for consistency and interpretability, then expanded into multiplicative polynomial features up to a chosen degree. Candidate terms are filtered by their interaction order (e.g., pairwise vs. higher-order interactions), after which a greedy, vectorized forward-selection routine identifies the subset of features that improve cross-validated log-space R² scores. The final model is fit using a GPU-backed **TorchLinearRegression**, and selected polynomial indices are stored for use at inference. In `predict`, new inputs are remapped, polynomial features rebuilt, and log-space predictions are exponentiated and shifted back to the original scale. Helper methods handle feature filtering, batched candidate evaluation, and fast GPU-based cross-validation, while `_gpu_poly_features` constructs polynomial expansions with readable names. Together, this class provides a scalable, interpretable, and high-performance method for multiplicative polynomial modeling entirely on the GPU.


In [11]:
class VectorizedMAPS:
    """Vectorized Multiplicative Polynomial Selection (GPU: CuPy + Torch OLS)"""

    def __init__(self, max_degree=3, max_interaction_order=2, tolerance=1e-4, verbose=False):
        self.max_degree = int(max_degree)
        self.max_interaction_order = int(max_interaction_order)
        self.tolerance = float(tolerance)
        self.verbose = verbose
        self.selected_features = None
        self.final_model = None
        self.location_shift = 0.0
        self.quantile_remapper = None
        self._feature_names_all = None
        self._candidate_indices = None

    def fit(self, X, y):
        # move to GPU
        X_cp = cp.asarray(X)
        y_cp = cp.asarray(y).reshape(-1)

        # ensure positivity -> log space
        y_min = float(cp.min(y_cp))
        if y_min <= 0.0:
            self.location_shift = abs(y_min) + 1.0
            y_pos = y_cp + self.location_shift
        else:
            self.location_shift = 0.0
            y_pos = y_cp
        log_y_cp = cp.log(y_pos)

        # quantile remap (for interpretability) on GPU
        self.quantile_remapper = QuantileRemapper().fit(X_cp)
        X_remapped = self.quantile_remapper.transform(X_cp)

        # polynomial features on remapped variables (GPU)
        X_poly, feat_names = self._gpu_poly_features(X_remapped, self.max_degree, include_bias=False)
        self._feature_names_all = feat_names

        # filter by interaction order
        candidate_indices = self._filter_by_interaction_order(feat_names)
        if len(candidate_indices) == 0:
            candidate_indices = list(range(min(X_cp.shape[1], X_poly.shape[1])))
        self._candidate_indices = candidate_indices

        X_cand = X_poly[:, candidate_indices]

        # forward selection in log space
        self.selected_features = self._vectorized_forward_selection(X_cand, log_y_cp)

        # final fit in log space (selected poly -> log_y)
        if self.selected_features and len(self.selected_features) > 0:
            X_final = X_cand[:, self.selected_features]
        else:
            # fallback: use remapped main effects if nothing selected
            X_final = X_remapped

        lr = TorchLinearRegression(fit_intercept=True, device="cuda")
        lr.fit(X_final, log_y_cp)
        self.final_model = lr

        # store absolute poly indices for prediction-time selection
        if self.selected_features and len(self.selected_features) > 0:
            abs_idxs = [candidate_indices[i] for i in self.selected_features]
        else:
            abs_idxs = []
        self.final_model.selected_features = abs_idxs
        return self

    def predict(self, X):
        if self.final_model is None:
            return cp.zeros(len(X), dtype=float)

        X_cp = cp.asarray(X)

        # remap using fitted quantile mapper (fallback: identity)
        if self.quantile_remapper is None:
            X_remapped = X_cp
        else:
            X_remapped = self.quantile_remapper.transform(X_cp)

        # predict in log space, then invert
        if hasattr(self.final_model, "selected_features") and len(self.final_model.selected_features) > 0:
            X_poly, _ = self._gpu_poly_features(X_remapped, self.max_degree, include_bias=False)
            abs_idxs = self.final_model.selected_features
            if len(abs_idxs) > 0 and max(abs_idxs) < X_poly.shape[1]:
                X_sel = X_poly[:, abs_idxs]
                log_y_pred = self.final_model.predict(X_sel)     # NumPy
            else:
                log_y_pred = self.final_model.predict(X_remapped)
        else:
            log_y_pred = self.final_model.predict(X_remapped)

        y_pred = cp.exp(log_y_pred) - self.location_shift
        return y_pred

    # ------------ helpers ------------
    def _filter_by_interaction_order(self, feature_names):
        valid = []
        for i, name in enumerate(feature_names):
            var_matches = re.findall(r'x(\d+)', name)
            if len(set(var_matches)) <= self.max_interaction_order:
                valid.append(i)
        return valid

    def _vectorized_forward_selection(self, X_cand, y_cp):
        return _vectorized_forward_selection(X_cand, y_cp, self.tolerance, self.verbose)

    def _batch_evaluate_candidates(self, X_cand, log_y_cp, selected, remaining):
        scores = []
        X_sel = X_cand[:, selected] if selected else None
        for cand in remaining:
            X_test = X_cand[:, [cand]] if X_sel is None else cp.concatenate([X_sel, X_cand[:, [cand]]], axis=1)
            scores.append(self._fast_cv_score(X_test, log_y_cp, cv_folds=3))
        return cp.asarray(scores, dtype=float)

    def _cv_r2_gpu(self, X_cp, y_cp, k=3):
        return _fast_cv_score(X_cp, y_cp, k)

    @staticmethod
    def _gpu_poly_features(X_cp: cp.ndarray, degree: int, include_bias: bool = False):
        return gpu_poly_features(X_cp, degree, include_bias)

# Adaptive Method Selector

The **`VectorizedAdaptiveMethodSelector`** (AMS) orchestrates an automated, GPU-accelerated choice between **APS** (additive polynomial selection) and **MAPS** (multiplicative polynomial selection) to fit the most suitable model for a given dataset. In `fit`, it first runs a fast, CuPy-backed characteristic analysis that scores the data on four axes: **multiplicative evidence** (whether log-space linearization helps), **interaction strength** (how much higher-degree terms improve cross-validated R² over main effects), **multiplicative separability** (how well a sum of univariate log models explains the response compared with a joint model), and **dynamic range** of `y`. A transparent decision function then chooses APS, MAPS, or a brief **both-methods** trial if the evidence is ambiguous; the chosen path trains a GPU-native polynomial model using `TorchLinearRegression` under the hood and records the selected features for reproducible inference. The class exposes `predict` for seamless forecasting via the selected submodel, `get_metrics` for quick reporting, and helper routines (`_cv_r2_gpu`, `_gpu_poly_features`, and the three diagnostic tests) that keep all heavy lifting on the GPU to minimize transfers. In short, AMS is a vectorized controller that profiles your problem, selects the right polynomial paradigm, and delivers a high-performance fit with concise, interpretable signals about why that choice was made.


In [12]:
class VectorizedAdaptiveMethodSelector:
    """Adaptive Method Selector with GPU batch processing (CuPy + Torch OLS)."""

    def __init__(self, max_degree=3, max_interaction_order=2, tolerance=1e-4, verbose=False):
        self.max_degree = int(max_degree)
        self.max_interaction_order = int(max_interaction_order)
        self.tolerance = float(tolerance)
        self.verbose = verbose
        self.selected_method = None
        self.method_scores = {}
        self.final_model = None

    def fit(self, X, y):
        if self.verbose:
            print("=== Vectorized AMS Analysis (GPU) ===")

        self.method_scores = self._vectorized_analyze_characteristics(X, y)
        recommended_method, confidence = self._make_method_decision(self.method_scores)

        if self.verbose:
            print(f"Recommended: {recommended_method} (confidence: {confidence:.3f})")

        if recommended_method == 'APS':
            self.selected_method = 'APS'
            self.final_model = VectorizedAPS(self.max_degree, self.max_interaction_order, self.tolerance, False)
            self.final_model.fit(X, y)

        elif recommended_method == 'MAPS':
            self.selected_method = 'MAPS'
            self.final_model = VectorizedMAPS(self.max_degree, self.max_interaction_order, self.tolerance, False)
            self.final_model.fit(X, y)

        else:  # BOTH_TEST
            if self.verbose:
                print("Testing both methods (GPU)...")
            aps_model = VectorizedAPS(self.max_degree, self.max_interaction_order, self.tolerance, False)
            aps_model.fit(X, y)
            aps_pred = aps_model.predict(X)
            aps_score = self._r2_cp(cp.asarray(y), cp.asarray(aps_pred))

            maps_model = VectorizedMAPS(self.max_degree, self.max_interaction_order, self.tolerance, False)
            maps_model.fit(X, y)
            maps_pred = maps_model.predict(X)
            maps_score = self._r2_cp(cp.asarray(y), cp.asarray(maps_pred))

            if self.verbose:
                print(f"APS R²: {aps_score:.4f}, MAPS R²: {maps_score:.4f}")

            if aps_score > maps_score:
                self.selected_method = 'APS'
                self.final_model = aps_model
            else:
                self.selected_method = 'MAPS'
                self.final_model = maps_model

        return self

    def predict(self, X):
        if self.final_model is None:
            raise ValueError("Model must be fitted first")
        return self.final_model.predict(X)

    # ---------------- GPU characteristic analysis ----------------

    def _vectorized_analyze_characteristics(self, X, y):
        X_cp = cp.asarray(X)
        y_cp = cp.asarray(y).reshape(-1)

        scores = {}
        scores['multiplicative_evidence']   = self._vectorized_multiplicativity_test(X_cp, y_cp)
        scores['interaction_strength']      = self._vectorized_interaction_test(X_cp, y_cp)
        scores['multiplicative_separability']= self._vectorized_separability_test(X_cp, y_cp)
        ymax = float(cp.max(y_cp))
        ymin = float(cp.min(y_cp))
        scores['dynamic_range'] = ymax / max(ymin, 1e-8)
        return scores

    def _vectorized_multiplicativity_test(self, X_cp, y_cp):
        try:
            n = int(y_cp.size)
            if n < 20:
                return 0.5

            ymax = float(cp.max(y_cp))
            ymin = float(cp.min(y_cp))
            dynamic_range = ymax / max(ymin, 1e-8)
            if dynamic_range < 3.0:
                return 0.0

            n_bootstrap = 5
            n_samples = min(n, max(50, n // 2))

            rng = cp.random.default_rng(42)
            improvement_ratios = []

            for _ in range(n_bootstrap):
                idx = rng.integers(0, n, size=n_samples, endpoint=False)
                idx_cp = cp.asarray(idx)
                Xb = X_cp[idx_cp]
                yb = y_cp[idx_cp]

                # Original space OLS
                lr1 = TorchLinearRegression(fit_intercept=True, device="cuda")
                lr1.fit(Xb, yb)
                yhat1 = lr1.predict(Xb)  # numpy
                yb_cp = cp.asnumpy(yb)
                res1 = cp.mean((yb_cp - yhat1)**2)

                # Log space (only if all positive)
                if float(cp.min(yb)) > 0.0:
                    log_yb = cp.log(yb)
                    lr2 = TorchLinearRegression(fit_intercept=True, device="cuda")
                    lr2.fit(Xb, log_yb)
                    log_pred = lr2.predict(Xb)          # numpy
                    yhat2 = cp.exp(log_pred)
                    res2 = cp.mean((yb_cp - yhat2)**2)
                    ratio = res1 / max(res2, 1e-8)
                else:
                    ratio = 0.5

                improvement_ratios.append(ratio)

            ratios = cp.asarray(improvement_ratios, dtype=float)
            med = float(cp.median(ratios))
            std = float(cp.std(ratios))
            if std > 0.5:
                med *= 0.8

            if   med > 3.0: mult = 0.9
            elif med > 2.0: mult = 0.8
            elif med > 1.5: mult = 0.6
            elif med < 0.6: mult = 0.1
            elif med < 0.8: mult = 0.3
            else:           mult = 0.4

            if dynamic_range > 20:
                mult = min(1.0, mult + 0.15)
            elif dynamic_range > 10:
                mult = min(1.0, mult + 0.10)
            return mult

        except Exception:
            return 0.5

    def _vectorized_interaction_test(self, X_cp, y_cp):
        try:
            # degree 1 (main effects) vs full poly up to max_degree
            X_main, _ = self._gpu_poly_features(X_cp, degree=1, include_bias=False)
            X_full, _ = self._gpu_poly_features(X_cp, degree=self.max_degree, include_bias=False)

            r2_main = self._cv_r2_gpu(X_main, y_cp, k=min(5, max(2, int(y_cp.size)//10)))
            r2_full = self._cv_r2_gpu(X_full, y_cp, k=min(5, max(2, int(y_cp.size)//10)))

            r2_main = max(0.0, r2_main)
            r2_full = max(0.0, r2_full)
            if r2_full <= r2_main:
                return 0.0

            return float(min(1.0, (r2_full - r2_main) / max(1.0 - r2_main, 1e-8)))
        except Exception:
            return 0.5

    def _vectorized_separability_test(self, X_cp, y_cp):
        try:
            ymin = float(cp.min(y_cp))
            if ymin <= 0.0:
                y_pos = y_cp - ymin + 1.0
            else:
                y_pos = y_cp
            log_y = cp.log(y_pos)

            d = int(X_cp.shape[1])
            if d < 2:
                return 0.0

            # Univariate fits (degree up to 2)
            preds = []
            for j in range(d):
                Xj = X_cp[:, [j]]
                Xj_poly, _ = self._gpu_poly_features(Xj, degree=min(2, self.max_degree), include_bias=False)
                lr = TorchLinearRegression(fit_intercept=True, device="cuda")
                lr.fit(Xj_poly, log_y)
                pj = lr.predict(Xj_poly)  # numpy
                preds.append(pj)

            sum_uni = cp.sum(preds, axis=0)
            # center to avoid overcounting intercepts
            sum_uni = sum_uni - (d - 1) * float(cp.mean(cp.asnumpy(log_y)))
            r2_sep = self._r2_cp(cp.asnumpy(log_y), sum_uni)

            # Full multivariate log fit up to degree 2 (to be conservative)
            X_full, _ = self._gpu_poly_features(X_cp, degree=min(2, self.max_degree), include_bias=False)
            lr_full = TorchLinearRegression(fit_intercept=True, device="cuda")
            lr_full.fit(X_full, log_y)
            pred_full = lr_full.predict(X_full)
            r2_full = self._r2_cp(cp.asnumpy(log_y), pred_full)

            if r2_full <= 0.1:
                return 0.0
            return float(min(1.0, r2_sep / max(r2_full, 1e-8)))
        except Exception:
            return 0.5

    # ---------------- decision & metrics ----------------

    def _make_method_decision(self, scores):
        mult_evidence = scores['multiplicative_evidence']
        interaction   = scores['interaction_strength']
        separability  = scores['multiplicative_separability']
        dynamic_range = scores['dynamic_range']
        
        # More reasonable thresholds for MAPS selection
        if mult_evidence > 0.55:  # Lowered from 0.7
            return 'MAPS', mult_evidence
        
        if mult_evidence < 0.25:  # Lowered from 0.35
            return 'APS', 1 - mult_evidence
        
        # Consider dynamic range and separability with lower thresholds
        if dynamic_range > 10 and separability > 0.75:  # Much more reasonable
            return 'MAPS', separability * 0.8
        
        # Strong separability indicator
        if separability > 0.85 and dynamic_range > 5:  # Lowered thresholds
            return 'MAPS', separability
        
        # Multiplicative evidence in middle range
        if mult_evidence > 0.4:  # Give MAPS more consideration
            if separability > 0.7 or dynamic_range > 8:
                return 'MAPS', mult_evidence
        
        # Low interaction strength suggests simpler model
        if interaction < 0.3:
            return 'APS', 1 - interaction
        
        # Default to testing both methods for borderline cases
        return 'BOTH_TEST', 0.5

    def get_metrics(self):
        if self.final_model is None:
            return {"method": "VectorizedAMS", "n_features": 0, "r2": 0, "aic": 0, "score": 0}
        n_features = len(getattr(self.final_model, 'selected_features', []))
        return {"method": f"VectorizedAMS->{self.selected_method}", "n_features": n_features, "r2": 0, "aic": 0, "score": 0}

    # ---------------- small helpers ----------------

    def _cv_r2_gpu(self, X_cp, y_cp, k=3):
        n = int(X_cp.shape[0])
        if int(X_cp.shape[1]) >= 0.8*n or n < 15:
            return -10.0
    
        # Fix: use k parameter instead of undefined cv_folds
        k = int(min(k, max(2, n // 10)))
        
        # simple KFold indices
        rng = cp.random.default_rng(42)
        idx = rng.permutation(n)       # cp.ndarray of [0..n-1] permuted on GPU
        folds = cp.array_split(idx, k)
    
        r2s = []
        for i in range(k):
            val_idx = folds[i]
            trn_idx = cp.concatenate([folds[j] for j in range(k) if j != i])
            X_tr = X_cp[cp.asarray(trn_idx)]
            y_tr = y_cp[cp.asarray(trn_idx)]
            X_va = X_cp[cp.asarray(val_idx)]
            y_va = y_cp[cp.asarray(val_idx)]
    
            lr = TorchLinearRegression(fit_intercept=True, device="cuda")
            lr.fit(X_tr, y_tr)
            y_hat = lr.predict(X_va)      # numpy
            r2 = self._r2_cp(cp.asnumpy(y_va), y_hat)
            if cp.isfinite(r2):
                r2s.append(r2)
        return float(cp.mean(r2s)) if r2s else -10.0

    @staticmethod
    def _r2_cp(y_true, y_pred):
        y_true = cp.asarray(y_true).reshape(-1)
        y_pred = cp.asarray(y_pred).reshape(-1)
        ybar = y_true.mean()
        ss_res = cp.sum((y_true - y_pred)**2)
        ss_tot = cp.sum((y_true - ybar)**2)
        return 1.0 - ss_res / (ss_tot + 1e-12)
    @staticmethod
    def _gpu_poly_features(X_cp: cp.ndarray, degree: int, include_bias: bool = False):
        return gpu_poly_features(X_cp, degree, include_bias)



# Coefficient Recovery

The `evaluate_marginal_coefficient_recovery` function is designed to test how accurately a GPU-accelerated APS or MAPS model can recover known coefficients when restricted to main effects. It first builds a simplified version of the chosen model (`VectorizedAPS` or `VectorizedMAPS`) with only single-variable terms and fits it on the provided training data. A helper routine reconstructs polynomial feature names (linear, quadratic, cubic, and binary indicators) so the fitted coefficients can be mapped back to meaningful variable terms. The function then extracts these coefficients, compares them against the supplied ground-truth values, and computes several evaluation metrics: **sign accuracy** (how often the coefficient sign matches), **magnitude correlation** (correlation between true and recovered magnitudes, with a fallback if only one coefficient is available), and **coefficient RMSE** (the root mean squared error across matched terms). All computations are GPU-accelerated with CuPy, and the function is built to gracefully handle missing terms, NaNs, or fitting errors by returning default values with infinite RMSE. This provides a compact yet rigorous way to assess how well APS/MAPS captures marginal effects in practice.


In [13]:
def evaluate_marginal_coefficient_recovery(
    ams_model, X_train, y_train, true_coeffs, n_continuous, n_binary
):
    """
    GPU version: fits a main-effects-only model using your GPU APS/MAPS
    and compares recovered coefficients to the known 'true' ones.
    No sklearn; uses the same polynomial naming scheme as the GPU code.
    """
    if ams_model.final_model is None:
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': cp.inf,
            'n_compared': 0
        }

    # helper to mirror GPU poly naming (degree up to 3)
    def _poly_feature_names(d, degree):
        names = []
        # degree 1
        for j in range(d):
            names.append(f"x{j}")
        if degree >= 2:
            for j in range(d):
                names.append(f"x{j}^2")
            for a in range(d):
                for b in range(a+1, d):
                    names.append(f"x{a} x{b}")
        if degree >= 3:
            for j in range(d):
                names.append(f"x{j}^3")
            for a in range(d):
                for b in range(d):
                    if b == a: 
                        continue
                    names.append(f"x{a}^2 x{b}")
            for a in range(d):
                for b in range(a+1, d):
                    for c in range(b+1, d):
                        names.append(f"x{a} x{b} x{c}")
        return names

    try:
        # Build a main-effects-only GPU model of the same family
        if ams_model.selected_method == 'APS':
            coeff_model = VectorizedAPS(
                max_degree=ams_model.max_degree,
                max_interaction_order=1,   # main effects only
                tolerance=ams_model.tolerance,
                verbose=False
            )
        else:  # MAPS
            coeff_model = VectorizedMAPS(
                max_degree=ams_model.max_degree,
                max_interaction_order=1,   # main effects only
                tolerance=ams_model.tolerance,
                verbose=False
            )

        # Fit on GPU (your GPU classes handle CuPy/Torch internally)
        coeff_model.fit(X_train, y_train)
        inner_model = coeff_model.final_model

        # Map selected poly indices -> names for parsing
        d_total = n_continuous + n_binary
        all_feature_names = _poly_feature_names(d_total, ams_model.max_degree)

        fitted_coeffs = {}

        # Intercept (TorchLinearRegression exposes .intercept_)
        if hasattr(inner_model, 'intercept_'):
            fitted_coeffs['intercept'] = float(inner_model.intercept_)

        selected_features = list(getattr(inner_model, 'selected_features', []) or [])
        model_coeffs = cp.asarray(getattr(inner_model, 'coef_', []), dtype=float)

        if selected_features and model_coeffs.size:
            for i, feat_idx in enumerate(selected_features):
                if i >= model_coeffs.size or feat_idx >= len(all_feature_names):
                    continue
                feature_name = all_feature_names[feat_idx]
                coeff_value = float(model_coeffs[i])

                # Only keep main effects (single unique variable in the term)
                var_matches = re.findall(r'x(\d+)', feature_name)
                unique_vars = set(var_matches)

                if len(unique_vars) <= 1:
                    if '^2' in feature_name and ' ' not in feature_name:
                        # quadratic of a continuous variable
                        var_idx = int(feature_name.split('^')[0][1:])
                        if var_idx < n_continuous:
                            fitted_coeffs[f'x{var_idx}_quadratic'] = coeff_value
                    elif '^3' in feature_name and ' ' not in feature_name:
                        # cubic of a continuous variable
                        var_idx = int(feature_name.split('^')[0][1:])
                        if var_idx < n_continuous:
                            fitted_coeffs[f'x{var_idx}_cubic'] = coeff_value
                    else:
                        # linear/binary main effect
                        if feature_name.startswith('x') and '^' not in feature_name and ' ' not in feature_name:
                            var_idx = int(feature_name[1:])
                            if var_idx < n_continuous:
                                fitted_coeffs[f'x{var_idx}_linear'] = coeff_value
                            else:
                                fitted_coeffs[f'x{var_idx}_binary'] = coeff_value

        # Compare to true coefficients
        common = [k for k in true_coeffs.keys() if k in fitted_coeffs]
        if not common:
            return {
                'sign_accuracy': 0.0,
                'magnitude_correlation': 0.0,
                'coefficient_rmse': cp.inf,
                'n_compared': 0
            }

        true_vals = cp.array([true_coeffs[k] for k in common], dtype=float)
        fit_vals  = cp.array([fitted_coeffs[k] for k in common], dtype=float)

        # drop any NaNs from true values
        mask = ~cp.isnan(true_vals)
        true_vals = true_vals[mask]
        fit_vals  = fit_vals[mask]

        if true_vals.size == 0:
            return {
                'sign_accuracy': 0.0,
                'magnitude_correlation': 0.0,
                'coefficient_rmse': cp.inf,
                'n_compared': 0
            }

        sign_accuracy = float(cp.mean(cp.sign(true_vals) == cp.sign(fit_vals)))

        if true_vals.size > 1:
            mag_corr = cp.corrcoef(cp.abs(true_vals), cp.abs(fit_vals))[0, 1]
            magnitude_corr = float(0.0 if cp.isnan(mag_corr) else mag_corr)
        else:
            rel_err = abs(true_vals[0] - fit_vals[0]) / (abs(true_vals[0]) + 1e-10)
            magnitude_corr = float(max(0.0, 1.0 - rel_err))

        coefficient_rmse = float(cp.sqrt(cp.mean((true_vals - fit_vals) ** 2)))

        return {
            'sign_accuracy': sign_accuracy,
            'magnitude_correlation': magnitude_corr,
            'coefficient_rmse': coefficient_rmse,
            'n_compared': int(true_vals.size)
        }

    except Exception:
        return {
            'sign_accuracy': 0.0,
            'magnitude_correlation': 0.0,
            'coefficient_rmse': cp.inf,
            'n_compared': 0
        }


# Design of Experiments (Half Fraction)

This module defines a GPU-accelerated pipeline for **design of experiments (DOE)** and benchmarking interpretable AMS models against black-box baselines. The core generator, `generate_vectorized_half_fraction_design`, creates a set of experimental configurations by combining factorial design matrices (built with `_generate_resolution_iv_design` on the GPU) with different factor levels (continuous, binary, polynomial degree, interaction structures, noise levels, etc.). Invalid setups are filtered using `_is_valid_config`. Each valid configuration is replicated multiple times to form the full DOE. The orchestrator, `run_interpretable_experiments`, then executes the workflow in three stages: (1) **data generation** via `VectorizedBinaryPolynomialFunction` (CuPy arrays produced entirely on GPU), (2) **benchmarking** where interpretable AMS models compete against black-box models in batches, and (3) **analysis** of champion results summarized in a GPU dataframe. Helper routines like `_batch_process_experiments` streamline batch execution, while GPU design construction ensures scalability. Together, these functions allow systematic evaluation of whether AMS can remain interpretable while outperforming traditional black-box approaches.


In [14]:
def generate_vectorized_half_fraction_design(function_type=None, n_replications=3):
    """Generate half-fraction design (design matrix on GPU via CuPy)."""
    # Function types
    function_types = [function_type] if function_type else ['additive', 'multiplicative', 'mixed']

    # Factor levels
    factors = {
        'n_continuous': [4, 6],
        'n_binary': [0, 2],
        'degree': [2, 3],
        'interaction_order': [1, 2],
        'noise_level': [0.2, 0.3],
        'binary_effect_strength': [0.5, 1.5],
        'n_samples': [500, 1000]
    }
    interaction_patterns = [
        ['continuous'],
        ['binary'],
        ['continuous', 'binary', 'mixed'],
        ['mixed']
    ]
    factor_names = list(factors.keys())

    # GPU design matrix
    design_matrix = _generate_resolution_iv_design(len(factor_names))  # cp.ndarray

    print(f"=== VECTORIZED HALF-FRACTION DESIGN ===")
    print(f"Function types: {len(function_types)}")
    print(f"Base design matrix: {int(design_matrix.shape[0])} runs x {int(design_matrix.shape[1])} factors")
    print(f"Interaction patterns: {len(interaction_patterns)}")
    print(f"Replications per base experiment: {n_replications}")

    base_experiments = []
    base_experiment_id = 0

    for func_type in function_types:
        print(f"  Processing function type: {func_type}")
        # Iterate rows from GPU; pull scalar bits with .item()
        for run_idx in range(int(design_matrix.shape[0])):
            factor_levels = design_matrix[run_idx]  # cp array row
            for interaction_pattern in interaction_patterns:
                base_experiment_id += 1

                config = {
                    'base_experiment_id': base_experiment_id,
                    'function_type': func_type,
                    'interaction_types': interaction_pattern
                }

                for i, factor_name in enumerate(factor_names):
                    level_index = int(factor_levels[i].item())
                    config[factor_name] = factors[factor_name][level_index]

                config['total_features'] = config['n_continuous'] + config['n_binary']

                if _is_valid_config(config):
                    base_experiments.append(config)

    print(f"  Generated {len(base_experiments)} valid base experiments")

    # Distribution by function type
    func_type_counts = {}
    for exp in base_experiments:
        ft = exp['function_type']
        func_type_counts[ft] = func_type_counts.get(ft, 0) + 1
    print(f"  Function type distribution in base: {func_type_counts}")

    # Replicate
    all_experiments = []
    experiment_id = 0
    for base_exp in base_experiments:
        for rep in range(n_replications):
            experiment_id += 1
            replicated_exp = base_exp.copy()
            replicated_exp['experiment_id'] = experiment_id
            replicated_exp['replication'] = rep
            all_experiments.append(replicated_exp)

    # Final checks
    final_func_type_counts = {}
    for exp in all_experiments:
        ft = exp['function_type']
        final_func_type_counts[ft] = final_func_type_counts.get(ft, 0) + 1

    print(f"=== FINAL DESIGN SUMMARY ===")
    print(f"Base experiments: {len(base_experiments)}")
    print(f"Total experiments with replications: {len(all_experiments)}")
    print(f"Expected total: {len(base_experiments)} × {n_replications} = {len(base_experiments) * n_replications}")
    print(f"Final function type distribution: {final_func_type_counts}")

    for ft in function_types:
        count = final_func_type_counts.get(ft, 0)
        expected = func_type_counts.get(ft, 0) * n_replications
        print(f"  {ft}: {count} experiments ({count // n_replications} base × {n_replications} reps)")
        if count != expected:
            print(f"    ⚠️  WARNING: Expected {expected}, got {count}")

    return all_experiments


def _generate_resolution_iv_design(n_factors: int) -> cp.ndarray:
    """
    Generate a Resolution IV half-fraction design matrix on GPU (CuPy).
    Returns a cp.ndarray of shape (runs, n_factors) with {0,1} entries.
    """
    if n_factors <= 4:
        # Full factorial: all 2^k combinations using GPU bit tricks
        runs = 1 << n_factors
        idx = cp.arange(runs, dtype=cp.uint32)[:, None]                         # (runs, 1)
        bitpos = cp.arange(n_factors, dtype=cp.uint32)[None, :]                 # (1, n_factors)
        mat = ((idx >> bitpos) & 1).astype(cp.int8)                             # (runs, n_factors)
        return mat

    # Base 4-factor (A,B,C,D) 2^4 = 16 runs
    base_runs = 1 << 4
    idx = cp.arange(base_runs, dtype=cp.uint32)[:, None]                        # (16,1)
    bitpos = cp.arange(4, dtype=cp.uint32)[None, :]                             # (1,4)
    base = ((idx >> bitpos) & 1).astype(cp.int8)                                # (16,4) -> A,B,C,D

    # Generators (Resolution IV): E = A⊕B⊕C, F = B⊕C⊕D, G = A⊕C⊕D
    A, B, C, D = base[:, 0], base[:, 1], base[:, 2], base[:, 3]
    E = (A ^ B ^ C).astype(cp.int8)
    F = (B ^ C ^ D).astype(cp.int8)
    G = (A ^ C ^ D).astype(cp.int8)

    full = cp.concatenate([base, E[:, None], F[:, None], G[:, None]], axis=1)   # (16,7)

    # If more than 7 factors requested, pad with zeros; else slice to n_factors
    if n_factors <= 7:
        return full[:, :n_factors]
    else:
        zeros = cp.zeros((full.shape[0], n_factors - full.shape[1]), dtype=cp.int8)
        return cp.concatenate([full, zeros], axis=1)


def _is_valid_config(config) -> bool:
    """Validate experiment configuration (CPU logic — negligible cost)."""
    n_continuous = config['n_continuous']
    n_binary = config['n_binary']
    interaction_types = config['interaction_types']
    degree = config['degree']

    if n_continuous == 0 and n_binary == 0:
        return False

    if n_binary == 0 and any(itype in ['binary', 'mixed'] for itype in interaction_types):
        return False
    if n_continuous < 2 and 'continuous' in interaction_types:
        return False
    if n_binary < 2 and 'binary' in interaction_types:
        return False

    total_features = n_continuous + n_binary
    if total_features > 8 or (total_features > 6 and degree > 2):
        return False

    return True


def run_interpretable_experiments(function_type=None):
    """Top-level orchestration (mostly control flow/IO; heavy math already on GPU elsewhere)."""
    print("=== INTERPRETABLE MODEL vs BLACK BOX BENCHMARKS DOE ===")

    experiments = generate_vectorized_half_fraction_design(function_type)

    print(f"\nExecuting {len(experiments)} experiments...")

    start_time = time.time()

    # STAGE 1: Data Generation (your VectorizedBinaryPolynomialFunction now emits CuPy on-GPU arrays)
    print("\n🔄 STAGE 1: Batch Data Generation")
    batch_data = VectorizedBinaryPolynomialFunction.generate_batch_mixed_data(experiments)

    if len(batch_data) == 0:
        print("❌ No valid data generated")
        return cudf.DataFrame()

    print(f"✅ Generated {len(batch_data)} datasets")

    # STAGE 2: Interpretable Model Evaluation
    print("\n🏆 STAGE 2: Interpretable Model vs Black Box Evaluation")
    # Check memory and adapt batch size
    free_mem = gpu_memory_check("Stage 2 Memory Check")
    
    if free_mem > 12000:      # >12GB free
        batch_size = min(100, len(batch_data) // 2)
    elif free_mem > 8000:     # >8GB free  
        batch_size = min(75, len(batch_data) // 3)
    elif free_mem > 5000:     # >5GB free
        batch_size = min(50, len(batch_data) // 4)
    else:                     # <5GB free
        batch_size = min(25, len(batch_data) // 6)
        print(f"⚠️ Low memory detected - using small batches of {batch_size}")
    
    print(f"Using batch size: {batch_size} (based on {free_mem:.0f}MB available)")
    results = []

    batch_size = min(100, len(batch_data) // 2)  # Larger batches, fewer iterations
    n_batches = (len(batch_data) + batch_size - 1) // batch_size

    for batch_idx in range(n_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(batch_data))
        batch = batch_data[start_idx:end_idx]

        print(f"  Processing batch {batch_idx+1}/{n_batches} ({len(batch)} model comparisons)")

        batch_results = _process_single_batch_champion_focused(batch)
        results.extend(batch_results)

    # STAGE 3: Interpretable ModelAnalysis
    print("\n📊 STAGE 3: Interpretable Model Results Analysis")

    if len(results) == 0:
        print("❌ No interpretable model estimates completed")
        return cudf.DataFrame()

    df = cudf.DataFrame(results)
    total_time = time.time() - start_time

    print(f"\n{'='*80}")
    print("INTERPRETABLE MODEL DOE RESULTS")
    print(f"{'='*80}")
    print(f"🏆 Completed {len(df)} model comparisons")
    print(f"⏱️  Total runtime: {total_time/60:.1f} minutes")

    analyzed_df = analyze_interpretable_champion_results(df)
    return analyzed_df


def _batch_process_experiments(batch_data):
    results = []
    batch_size = 50
    n_batches = (len(batch_data) + batch_size - 1) // batch_size

    for batch_idx in range(n_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(batch_data))
        batch = batch_data[start_idx:end_idx]

        print(f"  Processing batch {batch_idx+1}/{n_batches} ({len(batch)} experiments)")
        batch_results = _process_single_batch(batch)
        results.extend(batch_results)

    return results


# Evaluator

This module provides a GPU-oriented evaluation framework for comparing an interpretable model (AMS) against GPU baselines (cuML Random Forest and Torch MLP), with all computations performed on the GPU when possible. It is designed to be efficient, neutral in its messaging, and compatible with both pandas and cuDF data structures.

The evaluator integrates CuPy as the default GPU backend, falling back to NumPy if no GPU is available. Zero-copy bridges are implemented between CuPy and Torch via `dlpack`, allowing efficient sharing of GPU memory. An on-GPU R² scoring function (`cp_r2_score`) is provided to evaluate model predictions directly on CuPy arrays, with safeguards against division by zero when variance is low. A simple Torch-based multilayer perceptron (MLP) regressor is also included, supporting GPU training with adjustable parameters such as batch size, epochs, learning rate, and weight decay. The regressor includes `fit()` and `predict()` methods that convert data seamlessly between CuPy and Torch.

At the core of the system is the `ModelEvaluator` class, which compares the interpretable AMS model with GPU baselines. For AMS, it reports R² scores, overfitting measures, and interpretability metrics such as coefficient sign accuracy, correlation, and RMSE. For the GPU baselines, it evaluates both cuML Random Forest and the Torch MLP, computing benchmark statistics including maximum and mean R² values. The evaluator then provides a neutral comparative analysis, determining whether AMS outperforms, matches, or lags behind the baselines. Interpretability is assessed based on correlation thresholds, and each evaluation is assigned a status level ranging from STRONG and COMPETITIVE to NEAR, INTERPRETABLE_ONLY, or WEAK. For backward compatibility, the `InterpretableChampionEvaluator` alias is maintained.

The framework also supports batch evaluations through the `_process_single_batch_eval` function, which performs GPU-based train/test splits, fits AMS models, retrieves true coefficients, and runs evaluations using the `ModelEvaluator`. Metadata such as method scores, feature counts, and interaction strengths are included in the results. A backward-compatible alias `_process_single_batch_champion_focused` is provided. Results are then analyzed through the `analyze_results` function, which summarizes distributions of status levels and selected methods, along with mean scores for AMS R², benchmark R², and interpretability. This analysis works with both pandas and cuDF data and is accessible through a compatibility wrapper named `analyze_interpretable_champion_results`.

In summary, this evaluator offers a GPU-first, neutral, and extensible framework for training and comparing interpretable and black-box models. It measures predictive performance, assesses overfitting, evaluates interpretability through coefficient recovery, and delivers neutral status tags that indicate the competitiveness of the AMS model relative to GPU baselines.


In [15]:
# === GPU-first evaluator (cuML + PyTorch), neutral messaging ===


try:
    import cupy as cp
    _HAS_CUPY = True
except ImportError:
    import numpy as cp  # fallback to NumPy if no GPU
    _HAS_CUPY = False

# --- zero-copy bridges (Torch <-> CuPy) ---
import torch.utils.dlpack as dlpack

def cupy_to_torch(x_cp: cp.ndarray) -> torch.Tensor:
    """Convert CuPy array to PyTorch tensor (version-agnostic)"""
    try:
        # Try new method name first (CuPy >= 9.0)
        if hasattr(x_cp, 'to_dlpack'):
            return dlpack.from_dlpack(x_cp.to_dlpack())
        # Fall back to old method name (CuPy < 9.0)
        elif hasattr(x_cp, 'toDlpack'):
            return dlpack.from_dlpack(x_cp.toDlpack())
        else:
            raise AttributeError("CuPy array has no DLPack method")
    except Exception as e:
        print(f"DLPack conversion failed, using CPU fallback: {e}")
        # CPU fallback (slower but reliable)
        return torch.from_numpy(cp.asnumpy(x_cp)).cuda()

def torch_to_cupy(x_th: torch.Tensor) -> cp.ndarray:
    """Convert PyTorch tensor to CuPy array (version-agnostic)"""
    try:
        # Try new method name first
        if hasattr(cp, 'from_dlpack'):
            return cp.from_dlpack(dlpack.to_dlpack(x_th))
        # Fall back to old method name
        elif hasattr(cp, 'fromDlpack'):
            return cp.fromDlpack(dlpack.to_dlpack(x_th))
        else:
            raise AttributeError("CuPy has no DLPack method")
    except Exception as e:
        print(f"DLPack conversion failed, using CPU fallback: {e}")
        # CPU fallback
        return cp.asarray(x_th.detach().cpu().numpy())

# --- on-GPU R^2 (works on 1-D CuPy arrays) ---
def cp_r2_score(y_true, y_pred):
    # Robust, CuPy-native R^2 that matches the inline formula you debugged
    yt = cp.asarray(y_true, dtype=cp.float32).reshape(-1)
    yp = cp.asarray(y_pred, dtype=cp.float32).reshape(-1)
    if yt.shape != yp.shape:
        raise ValueError(f"cp_r2_score: shape mismatch: {yt.shape} vs {yp.shape}")
    if not cp.isfinite(yt).all() or not cp.isfinite(yp).all():
        return float('nan')  # don’t silently produce nonsense
    ybar   = cp.mean(yt)
    ss_res = cp.sum((yt - yp) ** 2)
    ss_tot = cp.sum((yt - ybar) ** 2)
    return float(1.0 - ss_res / (ss_tot + 1e-12))

# --- a small Torch MLP regressor (GPU) ---
class TorchMLPRegressor(nn.Module):
    def __init__(self, in_dim: int, hidden=(128, 64, 32), dropout=0.2):
        super().__init__()
        layers = []
        last = in_dim
        
        for i, h in enumerate(hidden):
            layers += [
                nn.Linear(last, h),
                nn.BatchNorm1d(h),  # ✅ Helps with training stability
                nn.ReLU(),
                nn.Dropout(dropout) if i < len(hidden)-1 else nn.Identity()
            ]
            last = h
        
        layers += [nn.Linear(last, 1)]
        self.net = nn.Sequential(*layers)
        
        # ✅ Better weight initialization
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def fit(self, X_cp: cp.ndarray, y_cp: cp.ndarray,
            epochs=200, batch_size=1024, lr=1e-3, weight_decay=1e-4,
            verbose=False, device=None):
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"

        X_th = cupy_to_torch(X_cp).float().to(device)
        y_th = cupy_to_torch(y_cp).float().view(-1, 1).to(device)

        ds = TensorDataset(X_th, y_th)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

        self.to(device)
        opt = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.MSELoss()

        try:
            self.train()
            for ep in range(epochs):
                ep_loss = 0.0
                for xb, yb in dl:
                    opt.zero_grad(set_to_none=True)
                    pred = self.net(xb)
                    loss = loss_fn(pred, yb)
                    loss.backward()
                    opt.step()
                    ep_loss += loss.item()  # ✅ More efficient than .detach().cpu()
                
                # Optional: periodic cleanup
                if ep % 50 == 0:
                    torch.cuda.empty_cache()
                    
        finally:
            # ✅ Always cleanup
            torch.cuda.empty_cache()
        
        return self

    @torch.no_grad()
    def predict(self, X_cp: cp.ndarray, device=None) -> cp.ndarray:
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.eval().to(device)
        X_th = cupy_to_torch(X_cp).float().to(device)
        pred = self.net(X_th).squeeze(1)
        return torch_to_cupy(pred)

# === Evaluator (neutral language) ===
class ModelEvaluator:
    """
    Compare the interpretable model (AMS) vs GPU baselines (cuML RF, Torch MLP)
    with all computations on GPU where possible.
    """
    def __init__(self, random_state: int = 42):
        self.random_state = random_state
        self.rf_model = None
        self.nn_model = None

    def evaluate_models(self, X_train, X_test, y_train, y_test,
                        ams_model, true_coeffs, feature_info):
        """
        X_*, y_*: CuPy arrays
        ams_model: your fitted VectorizedAPS/VectorizedMAPS wrapper
        """
        results = {}

        # 1) Interpretable model (AMS)
        ams_results = self._evaluate_ams(X_train, X_test, y_train, y_test,
                                         ams_model, true_coeffs, feature_info)
        results.update(ams_results)

        # 2) GPU baselines
        blackbox_results = self._evaluate_gpu_baselines(X_train, X_test, y_train, y_test)
        results.update(blackbox_results)

        # 3) Analysis
        analysis = self._analyze_against_benchmarks(results)
        results.update(analysis)

        return results

    def _evaluate_ams(self, X_train, X_test, y_train, y_test,
                      ams_model, true_coeffs, feature_info):
            # Time AMS predictions
        start_time = time.time()
        ams_pred_test = ams_model.predict(X_test)
        ams_pred_train = ams_model.predict(X_train)
        ams_predict_time = time.time() - start_time
        ams_pred_test = ams_model.predict(X_test)
        ams_pred_train = ams_model.predict(X_train)

        # Ensure CuPy
        if not isinstance(ams_pred_test, cp.ndarray):
            ams_pred_test = cp.asarray(ams_pred_test)
        if not isinstance(ams_pred_train, cp.ndarray):
            ams_pred_train = cp.asarray(ams_pred_train)

        ams_r2_test = cp_r2_score(y_test, ams_pred_test)
        ams_r2_train = cp_r2_score(y_train, ams_pred_train)
        ams_overfit = ams_r2_train - ams_r2_test

        # Time coefficient recovery
        start_time = time.time()
        coeff_recovery = evaluate_marginal_coefficient_recovery_fixed(
            ams_model, X_train, y_train, true_coeffs,
            feature_info['n_continuous'], feature_info['n_binary']
        )
        coeff_recovery_time = time.time() - start_time

        # Interpretability metrics (your GPU-ized function)
        coeff_recovery = evaluate_marginal_coefficient_recovery_fixed(
            ams_model, X_train, y_train, true_coeffs,
            feature_info['n_continuous'], feature_info['n_binary']
        )

        return {
            'ams_r2': ams_r2_test,
            'ams_r2_train': ams_r2_train,
            'ams_overfit': ams_overfit,
            'ams_method': ams_model.selected_method,

            # interpretability (unchanged schema)
            'ams_coeff_sign_accuracy': coeff_recovery['sign_accuracy'],
            'ams_coeff_magnitude_correlation': coeff_recovery['magnitude_correlation'],
            'ams_coeff_rmse': coeff_recovery['coefficient_rmse'],
            'ams_coeffs_recovered': coeff_recovery['n_compared'],

            # simple interpretability flags
            'ams_strong_interpretability': coeff_recovery['magnitude_correlation'] > 0.9,
            'ams_good_interpretability': coeff_recovery['magnitude_correlation'] > 0.7,
            'ams_interpretability_score': coeff_recovery['magnitude_correlation'],
        }

    def _evaluate_gpu_baselines(self, X_train, X_test, y_train, y_test):
        # --- cuML Random Forest ---
        rf_res = self._eval_rf_cuml(X_train, X_test, y_train, y_test)

        # --- Torch MLP (with simple on-GPU standardization) ---
        nn_res = self._eval_torch_mlp(X_train, X_test, y_train, y_test)

        rf_r2 = float(rf_res.get('rf_r2', float('nan')))
        nn_r2 = float(nn_res.get('nn_r2', float('nan')))
        
        benchmark_r2_max  = max(rf_r2, nn_r2)
        benchmark_r2_mean = (rf_r2 + nn_r2) / 2.0

        out = {}
        out.update(rf_res)
        out.update(nn_res)
        out.update({
            'benchmark_r2_max': benchmark_r2_max,
            'benchmark_r2_mean': benchmark_r2_mean,
            'benchmark_best_method': 'RF' if rf_res['rf_r2'] > nn_res['nn_r2'] else 'NN'
        })
        return out

    def _eval_rf_cuml(self, X_train, X_test, y_train, y_test):
        try:
            self.rf_model = cuRF(
                n_estimators=100,
                max_depth=10,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=self.random_state,
                n_streams=8,
            )
            self.rf_model.fit(X_train, y_train)
    
            # Predictions
            rf_pred_test  = self.rf_model.predict(X_test)
            rf_pred_train = self.rf_model.predict(X_train)
    
            # Ensure CuPy arrays for scoring
            rf_pred_test  = cp.asarray(rf_pred_test)
            rf_pred_train = cp.asarray(rf_pred_train)
    
            # R² using your cp_r2_score (same formula as NN)
            rf_r2        = cp_r2_score(y_test,  rf_pred_test)
            rf_r2_train  = cp_r2_score(y_train, rf_pred_train)
            rf_overfit   = float(rf_r2_train - rf_r2)
    
            return {
                'rf_r2': rf_r2,
                'rf_r2_train': rf_r2_train,
                'rf_overfit': rf_overfit,
                'rf_success': True,
            }
    
        except Exception as e:
            print(f"🚨 RF Eval: Exception caught: {type(e).__name__}: {e}")
            import traceback; traceback.print_exc()
            return {
                'rf_r2': float('nan'),
                'rf_r2_train': float('nan'),
                'rf_overfit': float('nan'),
                'rf_success': False,
            }
    
    

    def _eval_torch_mlp(self, X_train, X_test, y_train, y_test):
        try:
            # Ensure proper data types
            X_train = cp.ascontiguousarray(X_train.astype(cp.float32))
            X_test = cp.ascontiguousarray(X_test.astype(cp.float32))
            y_train = cp.ascontiguousarray(y_train.astype(cp.float32))
            y_test = cp.ascontiguousarray(y_test.astype(cp.float32))
    
            # Validate data
            if not cp.all(cp.isfinite(X_train)) or not cp.all(cp.isfinite(y_train)):
                return {
                    'nn_r2': -10.0,
                    'nn_r2_train': -10.0,
                    'nn_overfit': 0.0,
                    'nn_success': False
                }
    
            if X_train.shape[0] < 10:
                return {
                    'nn_r2': -10.0,
                    'nn_r2_train': -10.0,
                    'nn_overfit': 0.0,
                    'nn_success': False
                }
    
            # Force positivity (like MAPS does)
            y_min = float(cp.min(y_train))
            if y_min <= 0.0:
                shift = abs(y_min) + 1.0
                y_train_pos = y_train + shift
                y_test_pos = y_test + shift
            else:
                shift = 0.0
                y_train_pos = y_train
                y_test_pos = y_test
    
            # Train in log-space for multiplicative stability
            y_train_log = cp.log(y_train_pos)
    
            # Standardize features
            mu = cp.mean(X_train, axis=0)
            sigma = cp.std(X_train, axis=0)
            sigma = cp.where(sigma < 1e-6, 1.0, sigma)
            
            Xtr = (X_train - mu) / sigma
            Xte = (X_test - mu) / sigma
    
            # Create and train model
            in_dim = Xtr.shape[1]
            hidden_size = min(64, max(32, in_dim * 4))
            self.nn_model = TorchMLPRegressor(in_dim, hidden=(hidden_size, hidden_size//2))
            
            # Train on log-transformed targets
            self.nn_model.fit(
                Xtr, y_train_log, 
                epochs=250, 
                batch_size=min(512, max(64, X_train.shape[0]//4)), 
                lr=5e-4, 
                weight_decay=1e-5
            )
    
            # Predict in log-space, then back-transform
            log_pred_test = self.nn_model.predict(Xte)
            log_pred_train = self.nn_model.predict(Xtr)
            
            # Back-transform to original scale
            nn_pred_test = cp.exp(log_pred_test) - shift
            nn_pred_train = cp.exp(log_pred_train) - shift
    
            # Validate predictions
            if not cp.all(cp.isfinite(nn_pred_test)) or not cp.all(cp.isfinite(nn_pred_train)):
                return {
                    'nn_r2': -10.0,
                    'nn_r2_train': -10.0,
                    'nn_overfit': 0.0,
                    'nn_success': False
                }
    
            # Calculate R² on original scale
            nn_r2 = cp_r2_score(y_test, nn_pred_test)
            nn_r2_train = cp_r2_score(y_train, nn_pred_train)
            
            # Validate R² values
            if not cp.isfinite(nn_r2):
                nn_r2 = -10.0
            if not cp.isfinite(nn_r2_train):
                nn_r2_train = -10.0
    
            nn_overfit = float(nn_r2_train - nn_r2)
    
            return {
                'nn_r2': float(nn_r2),
                'nn_r2_train': float(nn_r2_train),
                'nn_overfit': nn_overfit,
                'nn_success': True
            }
    
        except Exception as e:
            return {
                'nn_r2': -10.0,
                'nn_r2_train': -10.0,
                'nn_overfit': 0.0,
                'nn_success': False
            }


    def _analyze_against_benchmarks(self, results: dict):
        ams_r2 = results['ams_r2']
        rf_r2 = results['rf_r2']
        nn_r2 = results['nn_r2']
        bmax = results['benchmark_r2_max']
        bmean = results['benchmark_r2_mean']
        interp = results['ams_interpretability_score']

        beats_rf = ams_r2 > rf_r2
        beats_nn = ams_r2 > nn_r2
        beats_best = ams_r2 > bmax
        beats_avg  = ams_r2 > bmean

        gap_vs_rf = ams_r2 - rf_r2
        gap_vs_nn = ams_r2 - nn_r2
        gap_vs_best = ams_r2 - bmax
        gap_vs_avg  = ams_r2 - bmean

        # neutral status tags
        interpretable_and_outperforms = (interp > 0.8) and (ams_r2 > bmax)
        interpretable_and_competitive = (interp > 0.8) and (gap_vs_best > -0.05)  # within 5%

        # simple status level
        if interp > 0.8 and gap_vs_best > 0.02:
            status_level = 'STRONG'
        elif interp > 0.8 and gap_vs_best > -0.02:
            status_level = 'COMPETITIVE'
        elif interp > 0.8 and gap_vs_best > -0.05:
            status_level = 'NEAR'
        elif interp > 0.8:
            status_level = 'INTERPRETABLE_ONLY'
        else:
            status_level = 'WEAK'

        # keep a neutral boolean; also include legacy key for compatibility
        success = status_level in ['STRONG', 'COMPETITIVE']

        return {
            'beats_rf': beats_rf,
            'beats_nn': beats_nn,
            'beats_best': beats_best,
            'beats_avg': beats_avg,

            'gap_vs_rf': gap_vs_rf,
            'gap_vs_nn': gap_vs_nn,
            'gap_vs_best': gap_vs_best,
            'gap_vs_avg': gap_vs_avg,

            'interpretable_and_outperforms': interpretable_and_outperforms,
            'interpretable_and_competitive': interpretable_and_competitive,

            'status_level': status_level,
            'success': success,

            # legacy compatibility fields (you can remove once you update downstream code)
            'ams_beats_rf': beats_rf,
            'ams_beats_nn': beats_nn,
            'ams_beats_best_blackbox': beats_best,
            'ams_beats_avg_blackbox': beats_avg,
            'ams_vs_rf_gap': gap_vs_rf,
            'ams_vs_nn_gap': gap_vs_nn,
            'ams_vs_best_blackbox_gap': gap_vs_best,
            'ams_vs_avg_blackbox_gap': gap_vs_avg,
            'champion_level': status_level,
            'champion_success': success,
        }

# Backward-compat class alias 
InterpretableChampionEvaluator = ModelEvaluator

# === Batch processing (GPU split + neutral logs); keep old name as alias ===

def _process_single_batch_eval(batch):
    """
    GPU-oriented batch loop with systematic memory cleanup.
    """
    batch_results = []
    
    # Initial cleanup
    gpu_cleanup(verbose=True)
    
    for i, (X, y_true, y_noisy, feature_info, config) in enumerate(batch):
        try:
            # Periodic cleanup every 10 experiments
            if i % 10 == 0 and i > 0:
                gpu_cleanup(verbose=True)
                gpu_memory_check(f"Batch progress {i}/{len(batch)}")
            
            n = X.shape[0]
            if n < 20:
                continue

            # GPU train/test split
            rs = cp.random.RandomState(42 + int(config['experiment_id']))
            idx = rs.permutation(n)
            split = int(n * 0.8)
            tr, te = idx[:split], idx[split:]

            X_train, X_test = X[tr], X[te]
            y_train, y_test = y_noisy[tr], y_noisy[te]

            # Fit AMS
            ams_model = VectorizedAdaptiveMethodSelector(
                max_degree=config['degree'],
                max_interaction_order=config['interaction_order'],
                verbose=False
            )
            ams_model.fit(X_train, y_train)

            # True coefficients for interpretability eval
            true_coeffs = VectorizedBinaryPolynomialFunction.get_true_marginal_coefficients_fixed(
                config['n_continuous'], config['n_binary'],
                config['binary_effect_strength'], config['function_type'],
                X_sample=X_train, y_sample=y_train
            )

            evaluator = ModelEvaluator(random_state=42 + int(config['experiment_id']))
            eval_results = evaluator.evaluate_models(
                X_train, X_test, y_train, y_test, ams_model, true_coeffs, feature_info
            )
            
            # ENHANCED REAL-TIME LOGGING WITH METHOD SELECTION
            if i % 10 == 0:
                ams_r2 = eval_results.get('ams_r2', 0)
                rf_r2 = eval_results.get('rf_r2', 0) 
                nn_r2 = eval_results.get('nn_r2', 0)
                status = eval_results.get('status_level', 'UNKNOWN')
                interp = eval_results.get('ams_interpretability_score', 0)
                method = eval_results.get('ams_method', 'UNKNOWN')  # ADD THIS
                
                print(f"    Exp {config['experiment_id']:3d} ({config['function_type']:4s}→{method:4s}): "
                      f"AMS={ams_r2:.3f} RF={rf_r2:.3f} NN={nn_r2:.3f} "
                      f"Status={status} Interp={interp:.3f}")

            # expected method
            expected_method = (
                'APS' if config['function_type'] == 'additive'
                else 'MAPS' if config['function_type'] == 'multiplicative'
                else 'EITHER'
            )
            ams_correct = (ams_model.selected_method == expected_method) or (expected_method == 'EITHER')

            result = dict(config)
            result.update(eval_results)
            result.update({
                'expected_method': expected_method,
                'ams_correct': ams_correct,
                'multiplicative_evidence': ams_model.method_scores['multiplicative_evidence'],
                'interaction_strength': ams_model.method_scores['interaction_strength'],
                'separability': ams_model.method_scores['multiplicative_separability'],
                'dynamic_range': ams_model.method_scores['dynamic_range'],
                'total_features': config['n_continuous'] + config['n_binary'],
                'has_binary': config['n_binary'] > 0,
                'is_mixed_vars': (config['n_continuous'] > 0) and (config['n_binary'] > 0),
                'n_train': int(split),
                'n_test': int(n - split),
            })
            batch_results.append(result)

            # Extra cleanup for memory-intensive mixed functions
            if config['function_type'] == 'mixed':
                del X_train, X_test, y_train, y_test, ams_model, eval_results
                gpu_cleanup()

        except Exception as e:
            print(f"    Error in experiment {config['experiment_id']}: {str(e)[:120]}...")
            # Force cleanup on error
            gpu_cleanup(verbose=True)
            continue

    # Final cleanup
    gpu_cleanup(verbose=True)
    return batch_results

# Backward-compat alias
def _process_single_batch_champion_focused(batch):
    return _process_single_batch_eval(batch)


# === Neutral result analysis (with compat wrapper) ===

# --- tiny helper so code works with both cuDF and pandas ---
def _iter_kv(s):
    """Yield (key, value) pairs from a pandas or cuDF Series without
    forcing a full DataFrame → pandas conversion."""
    # cuDF path
    if hasattr(s, "values_host"):  # cuDF Series
        keys = s.index.to_pandas().tolist()
        vals = s.values_host.tolist()
        return zip(keys, vals)
    # pandas path
    return s.items()

def analyze_results(df):
    """
    GPU-safe printing/summary. Works whether df is cuDF or pandas.
    """
    n = len(df)
    print("\n================ RESULTS SUMMARY ================")
    print(f"Rows: {n}")

    # Example 1: status_level distribution (this was failing)
    level_counts = df["status_level"].value_counts().sort_index()
    print("\nStatus level counts:")
    for level, count in _iter_kv(level_counts):
        pct = 100 * count / max(n, 1)
        print(f"  {str(level):16s}: {int(count)} ({pct:.1f}%)")

    # Example 2: method distribution
    if "ams_method" in df.columns:
        meth_counts = df["ams_method"].value_counts().sort_index()
        print("\nMethod selection counts:")
        for meth, count in _iter_kv(meth_counts):
            pct = 100 * count / max(n, 1)
            print(f"  {str(meth):16s}: {int(count)} ({pct:.1f}%)")

    # Example 3: simple means (cuDF/pandas both fine)
    for col in ["ams_r2", "benchmark_r2_max", "ams_interpretability_score"]:
        if col in df.columns:
            mean_val = float(df[col].mean())  # cuDF returns scalar-like; cast for safety
            print(f"Mean {col}: {mean_val:.4f}")

    return df


# Backward-compat wrapper (so your existing call names still work)
def analyze_interpretable_champion_results(df):
    return analyze_results(df)


# Additive Execution

In [16]:

# =============================================================================
# RUN 1: ADDITIVE FUNCTIONS ONLY
# =============================================================================
if __name__ == "__main__":
    import os
    import time
    
    print("=== RUN 1: ADDITIVE FUNCTIONS ===")
    gpu_memory_check("Initial")
    
    # Run additive only
    df_additive = run_interpretable_experiments(function_type="additive")
    
    # Save immediately - FIXED VERSION
    df_additive_pandas = df_additive.to_pandas()  # Convert to pandas first
    df_additive_pandas.to_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_additive.csv", index=False)  # ✅ Use pandas AND add filename
    print(f"✅ Additive results saved: {len(df_additive)} experiments")
    
    # Clean up
    del df_additive, df_additive_pandas  # Delete both versions
    gpu_cleanup(verbose=True)
    cp.get_default_memory_pool().free_all_blocks()
    gpu_memory_check("After additive cleanup")

=== RUN 1: ADDITIVE FUNCTIONS ===
GPU Memory Initial: 1437.6MB used, 14865.0MB free (16302.6MB total)
=== INTERPRETABLE MODEL vs BLACK BOX BENCHMARKS DOE ===
=== VECTORIZED HALF-FRACTION DESIGN ===
Function types: 1
Base design matrix: 16 runs x 7 factors
Interaction patterns: 4
Replications per base experiment: 3
  Processing function type: additive
  Generated 32 valid base experiments
  Function type distribution in base: {'additive': 32}
=== FINAL DESIGN SUMMARY ===
Base experiments: 32
Total experiments with replications: 96
Expected total: 32 × 3 = 96
Final function type distribution: {'additive': 96}
  additive: 96 experiments (32 base × 3 reps)

Executing 96 experiments...

🔄 STAGE 1: Batch Data Generation
🔄 Generating data for 96 experiments on GPU (CuPy)…
  Processing group 1/16
  Processing group 6/16
  Processing group 11/16
  Processing group 16/16
✅ Generated data for 96 valid experiments (GPU)
✅ Generated 96 datasets

🏆 STAGE 2: Interpretable Model vs Black Box Evaluatio

# Multiplicative Execution

In [17]:
if __name__ == "__main__":
    import os
    import time
    
    print("=== RUN 2: MULTIPLICATIVE FUNCTIONS ===")
    gpu_memory_check("Initial")
    
    # Run multiplicative only
    df_multiplicative = run_interpretable_experiments(function_type="multiplicative")
    
    # Save immediately
    df_multiplicative_pandas = df_multiplicative.to_pandas() 
    df_multiplicative_pandas.to_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_multiplicative.csv", index=False)
    print(f"✅ Multiplicative results saved: {len(df_multiplicative)} experiments")
    
    # Clean up
    del df_multiplicative
    gpu_cleanup(verbose=True)
    cp.get_default_memory_pool().free_all_blocks()
    gpu_memory_check("After multiplicative cleanup")

=== RUN 2: MULTIPLICATIVE FUNCTIONS ===
GPU Memory Initial: 3327.6MB used, 12975.0MB free (16302.6MB total)
=== INTERPRETABLE MODEL vs BLACK BOX BENCHMARKS DOE ===
=== VECTORIZED HALF-FRACTION DESIGN ===
Function types: 1
Base design matrix: 16 runs x 7 factors
Interaction patterns: 4
Replications per base experiment: 3
  Processing function type: multiplicative
  Generated 32 valid base experiments
  Function type distribution in base: {'multiplicative': 32}
=== FINAL DESIGN SUMMARY ===
Base experiments: 32
Total experiments with replications: 96
Expected total: 32 × 3 = 96
Final function type distribution: {'multiplicative': 96}
  multiplicative: 96 experiments (32 base × 3 reps)

Executing 96 experiments...

🔄 STAGE 1: Batch Data Generation
🔄 Generating data for 96 experiments on GPU (CuPy)…
  Processing group 1/16
  Processing group 6/16
  Processing group 11/16
  Processing group 16/16
✅ Generated data for 96 valid experiments (GPU)
✅ Generated 96 datasets

🏆 STAGE 2: Interpretabl

# Mixed Function Execution

In [18]:
if __name__ == "__main__":
    import os
    import time
    
    print("=== RUN 3: MIXED FUNCTIONS ===")
    gpu_memory_check("Initial")
    
    # Run mixed only
    df_mixed = run_interpretable_experiments(function_type="mixed")
    
    # Save immediately
    df_mixed_pandas = df_mixed.to_pandas()
    df_mixed_pandas.to_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_mixed.csv", index=False)
    print(f"✅ Mixed results saved: {len(df_mixed)} experiments")
    
    # Clean up
    del df_mixed
    gpu_cleanup(verbose=True)
    cp.get_default_memory_pool().free_all_blocks()
    gpu_memory_check("Final cleanup")


=== RUN 3: MIXED FUNCTIONS ===
GPU Memory Initial: 3329.6MB used, 12973.0MB free (16302.6MB total)
=== INTERPRETABLE MODEL vs BLACK BOX BENCHMARKS DOE ===
=== VECTORIZED HALF-FRACTION DESIGN ===
Function types: 1
Base design matrix: 16 runs x 7 factors
Interaction patterns: 4
Replications per base experiment: 3
  Processing function type: mixed
  Generated 32 valid base experiments
  Function type distribution in base: {'mixed': 32}
=== FINAL DESIGN SUMMARY ===
Base experiments: 32
Total experiments with replications: 96
Expected total: 32 × 3 = 96
Final function type distribution: {'mixed': 96}
  mixed: 96 experiments (32 base × 3 reps)

Executing 96 experiments...

🔄 STAGE 1: Batch Data Generation
🔄 Generating data for 96 experiments on GPU (CuPy)…
  Processing group 1/16
  Processing group 6/16
  Processing group 11/16
  Processing group 16/16
✅ Generated data for 96 valid experiments (GPU)
✅ Generated 96 datasets

🏆 STAGE 2: Interpretable Model vs Black Box Evaluation
GPU Memory St

# Combined Results

In [19]:
if __name__ == "__main__":
    import pandas as pd
    
    print("=== COMBINING ALL RESULTS ===")
    
    # Load all three CSV files
    df_add = pd.read_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_additive.csv")
    df_mult = pd.read_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_multiplicative.csv")
    df_mixed = pd.read_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_mixed.csv")
    
    # Combine
    results_df = pd.concat([df_add, df_mult, df_mixed], ignore_index=True)
    
    # Save final combined results
    results_df.to_csv("/mnt/c/users/lfult/OneDrive - bc.edu/desktop/aps/results_combined.csv", index=False)
    
    print(f"📊 FINAL COMBINED RESULTS:")
    print(f"  Additive: {len(df_add)} experiments")
    print(f"  Multiplicative: {len(df_mult)} experiments") 
    print(f"  Mixed: {len(df_mixed)} experiments")
    print(f"  Total: {len(results_df)} experiments")
    
    # Convert to cuDF for your analysis functions
    results_cudf = cudf.from_pandas(results_df)
    analyze_results(results_cudf)


=== COMBINING ALL RESULTS ===
📊 FINAL COMBINED RESULTS:
  Additive: 96 experiments
  Multiplicative: 96 experiments
  Mixed: 96 experiments
  Total: 288 experiments

================ RESULTS SUMMARY ================
Rows: 288

Status level counts:
  COMPETITIVE     : 14 (4.9%)
  INTERPRETABLE_ONLY: 5 (1.7%)
  NEAR            : 3 (1.0%)
  STRONG          : 252 (87.5%)
  WEAK            : 14 (4.9%)

Method selection counts:
  APS             : 82 (28.5%)
  MAPS            : 206 (71.5%)
Mean ams_r2: 0.8913
Mean benchmark_r2_max: 0.7766
Mean ams_interpretability_score: 0.9610
